In [ ]:
%matplotlib notebook

import numpy as np
import pylab as plt
plt.rc('text', usetex=True)
plt.rc('font', size=18,family='serif')
import matplotlib.gridspec as gridspec
from matplotlib.ticker import FormatStrFormatter
from emcee import EnsembleSampler
from emcee.backends import HDFBackend
from multiprocessing import cpu_count, Pool
from corner import corner
from Cosmology import *
import sys


#load autoreload, which automatically reloads the cosmology.py upon execution
%reload_ext autoreload
%autoreload 1
%aimport Cosmology


#Parameters for BAO and CMB
h = .6727
cLight=3E5
omega_baryon_preset = 0.02235/h**2
omega_gamma_preset = 2.469E-5/h**2

#Cosmological parameters
Omega_m_preset = 0.3166
#Omega_r_preset = 2.469E-5/h**2 # this is the photon density
Omega_c_preset = 0.6834

#Whether to use the numerical approximation for the comoving sound horizon, see arXiv:1411.1074:
#(we use rd analytical)
rd_num = False
#Whether to use the numerical approximation for the redshift of recombination,
#cf arXiv:1808.05724, or use z_d = z_star = 1089:
#(we use the numerical approx for z_star and z_d)
z_num = True

LCDM = cosmology(Omega_m_preset, Omega_c_preset, omegab=omega_baryon_preset, Hzero=100.*h)

In [ ]:
#Check versions of packages:

In [ ]:
#conda list matplotlib

In [ ]:
#conda list ipykernel

# Import data

## JLA Supernova data

In [ ]:
dataSN = np.loadtxt('data/jla_lcparams.txt', usecols=(2,4,6,8,10))
errSN = np.loadtxt('data/jla_lcparams.txt', usecols=(5,7,9))[np.argsort(dataSN.T[0])]
dataSN = dataSN[np.argsort(dataSN.T[0])]

#best fit values found in JLA analysis arXiv:1401.4064
a = 0.14
b = 3.14
MB = -19.04
delta_Mhost = -.06

In [ ]:
SNdata = Supernova_data(dataSN, errSN, np.array([a,b,MB, 0]))

## Quasar data

In [ ]:
dataQ = np.loadtxt('data/quasar_data_RL.txt', usecols=(0,1,2,3,4))
errQ = np.loadtxt('data/quasar_data_RL.txt', usecols=5)[np.argsort(dataQ.T[0])]
dataQ = dataQ[np.argsort(dataQ.T[0])]

#best fit values found in Risaliti & Lusso, Nature Astronomy, 2018
beta_prime, s = 7.4, 1.5

In [ ]:
Qdata = Quasar_data(dataQ, errQ, np.array([beta_prime, s]))

# BAO data

In [ ]:
#load BOSS data points
dataBAO = np.loadtxt('data/BOSS.txt', usecols=(1,3))
#load BOSS errors & measurement quantity
errBAO  = np.diag(np.loadtxt('data/BOSS.txt', usecols=4))
typeBAO = np.genfromtxt('data/BOSS.txt',dtype=str, usecols=2)
zdBAO = np.genfromtxt('data/BOSS.txt', usecols=6)

#BOSS DR12 covariance matrix from 1607.03155
sigmaDR12 = np.diag(np.diag(errBAO)[np.where(np.loadtxt('data/BOSS.txt', usecols=0, dtype=str) == 'BOSS_DR12')])
corrDR12 = np.tril(np.loadtxt('data/BOSS_DR12_cov.txt', usecols=(1,2,3,4,5,6))*1E-4)
corrDR12 += corrDR12.T
corrDR12 -= np.eye(len(corrDR12))
CovDR12 = np.dot(sigmaDR12, np.dot(corrDR12, sigmaDR12))

#eBOSS Quasar covariance matrix from 1801.03043
sigmaeBOSS = np.diag(np.diag(errBAO)[np.where(np.loadtxt('data/BOSS.txt', usecols=0, dtype=str) == 'eBOSS_QSO')])
correBOSS = np.triu(np.loadtxt('data/eBOSS_QSO_cov.txt', usecols=(1,2,3,4,5,6,7,8))*1E-4)
correBOSS += correBOSS.T
correBOSS -= np.eye(len(correBOSS[0]))
CoveBOSS = np.dot(sigmaeBOSS, np.dot(correBOSS, sigmaeBOSS))

#assemble the full covariance matrix
load_cov = np.diag([1 if i else 0 for i in np.loadtxt('data/BOSS.txt',usecols=5)==1]) 
CovBAO = (errBAO - np.dot(load_cov,errBAO))**2
#CovBAO = (errBAO)**2
CovBAO += np.pad(CovDR12,[(2,len(errBAO)-2-len(CovDR12)),(2,len(errBAO)-2-len(CovDR12))], mode='constant', constant_values=0)
CovBAO += np.pad(CoveBOSS,[(2 + len(CovDR12) + 1,len(errBAO)- 3 - len(CovDR12) - len(CoveBOSS)),(2 + len(CovDR12) + 1,len(errBAO)- 3 - len(CovDR12) - len(CoveBOSS))], mode='constant', constant_values=0)

#Finally, add the correlations given in 1904.03430 
add_cov = np.array([i if i<0 else 0 for i in np.loadtxt('data/BOSS.txt',usecols=5)])
Cov = np.pad(np.diag(add_cov[1:]), [(1,0),(0,1)], mode='constant', constant_values=0) + np.pad(np.diag(add_cov[1:]), [(0,1), (1,0)], mode='constant', constant_values=0)
for ind in np.array([np.where(add_cov<0)[0]-1., np.where(add_cov<0)[0]], dtype=int).T:
    Cov[ind] = Cov[ind] * np.diag(errBAO)[ind[0]] * np.diag(errBAO)[ind[1]]
    
CovBAO += Cov

BAOdata = BAO_data(dataBAO, CovBAO, typeBAO)

In [ ]:
# # load all data points except for WiggleZ
# dataBAO = np.loadtxt('data/BOSS.txt', usecols=(1,3))
# # dataBAO = np.loadtxt('data/bao_1806-06781.txt', usecols=(1,3))
# # add WiggleZ
# dataBAO = np.append(dataBAO, np.loadtxt('data/WiggleZ.txt', usecols=(1,3)), axis=0)

# # the error for BAO is a cov mat, due to the addition of the WiggleZ data.
# errBAO  = np.pad(np.diag(np.loadtxt('data/BOSS.txt', usecols=4)), [(0, 3), (0, 3)], mode='constant', constant_values=0)
# # errBAO  = np.pad(np.diag(np.loadtxt('data/bao_1806-06781.txt', usecols=4)), [(0, 3), (0, 3)], mode='constant', constant_values=0)

# # now add the WiggleZ cov mat. Note that this is the sqrt, as all the other errors are given in this format, too.
# errBAO += np.pad(np.sqrt(np.loadtxt('data/WiggleZ_cov.txt', usecols=(3,4,5))), [(len(errBAO)-3, 0), (len(errBAO)-3, 0)], mode='constant', constant_values=0)

# typeBAO = np.genfromtxt('data/BOSS.txt',dtype=str, usecols=2)
# # typeBAO = np.genfromtxt('data/bao_1806-06781.txt',dtype=str, usecols=2)
# typeBAO = np.append(typeBAO, np.genfromtxt('data/WiggleZ.txt',dtype=str, usecols=2), axis=0)

# RC data

In [ ]:
gamma00, kappa0 = 0.0093, 95
RCdata = RC_data([gamma00, kappa0])

# Planck 2018 CMB data

In [ ]:
# Planck18 distance priors are model dependent. Therefore they are defined in the model sections.

# Model 0: $\Lambda$LCDM

## which data sets to include?

In [ ]:
model = 'LCDM'
CMBdata = CMB_data(model, 'Planck18')

In [ ]:
#Define the data sets to use (from SNdata, Qdata, BAOdata, CMBdata)
data_sets = [SNdata]
data_set_names = [set.name for set in data_sets]
data_sets_str = '_' + '_'.join([name for name in data_set_names]) + '_' 

In [ ]:
## Define the likelihood and priors
ranges_LCDM_min=[0, 0, 60., -5, -10, -30, -.5, 0, 0]
ranges_LCDM_max=[1, 0.1, 80., 5, 10, -10, .5, 10, 3]
def lnprob_LCDM(theta):
    l = likelihood(theta, data_sets, ranges_LCDM_min, ranges_LCDM_max, model = 'LCDM', rd_num = rd_num, z_num = z_num)
    #l.get_settings() #returns the settings z_num, rd_num of the given likelihood
    #use only the flat priors with ranges defined above...
    #return l.logprobability_flat_prior()
    #... or add also the Gaussian prior from BBN:
    return l.logprobability_gauss_prior()

## define a reference BIC

In [ ]:
theta_LCDM = [Omega_m_preset, omega_baryon_preset, 67.27, a, b, MB, -0.05, beta_prime, s]

# calculates the BIC for LCDM: in this case, we compare to the preset values. The BIC is a function of the data set:
BIC_LCDM = lambda theta, data_sets_var: np.log(sum([len(d.get_data()) for d in data_sets_var])) * len(theta) -2*lnprob_LCDM(theta)

## run the MCMC sampler  [arXiv:1202.3665]...

In [ ]:
ndim, nwalkers, nsteps = 9, 512, 1000
pos0 = np.random.uniform(ranges_LCDM_min, ranges_LCDM_max, (nwalkers,len(ranges_LCDM_max)))

#pool = Pool(cpu_count())
#write = HDFBackend('chains/testing/LCDM' + data_sets_str + str(nwalkers) + 'x' + str(nsteps) + '.h5')
#sampler = EnsembleSampler(nwalkers, ndim, lnprob_LCDM, pool=pool)#, backend=write)

#sampler.run_mcmc(pos0, nsteps, progress=True);

## ...or load existing chains

In [ ]:
LCDM_sampler = HDFBackend('chains/LCDM' + data_sets_str + str(nwalkers) + 'x' + str(nsteps) + '.h5', read_only=True)
#LCDM_sampler = sampler

nsteps, nwalkers, ndim = LCDM_sampler.get_chain().shape

labs = [r'$\Omega_m$', r'$\Omega_b$', r'$H_0\ [\frac{\mathrm{km}/s}{\mathrm{Mpc}}]$', r'$\alpha$', r'$\beta$', r'$M_B$', r'$\Delta M_B$', r'$\beta^\prime$', r'$\delta$']

## Check and visualise the convergence: $\tau_f / n_\mathrm{steps} < 1/30$

In [ ]:
Nmin = 300
while Nmin < nsteps and np.all(LCDM_sampler.get_autocorr_time(tol=0,discard=Nmin)/(nsteps)*30 > 1):
    Nmin+=100
    
print('Nmin = {}'.format(Nmin))

In [ ]:
LCDM_samples = LCDM_sampler.get_chain()

# Delete walkers which haven't moved from initiatial position:

dellist = np.unique(np.where(np.isclose(LCDM_samples[-1] - LCDM_samples[0], 0))[0])
LCDM_samples = np.delete(LCDM_samples, np.unique(dellist), axis=1)

# plot the convergence:

f, ax = plt.subplots(len(LCDM_samples.T), 1, figsize=(6,3*len(LCDM_samples.T)))

for i in range(len(LCDM_samples.T)):
    ax[i].plot(LCDM_samples.T[i].T, lw=0.5)
    ax[i].plot([Nmin, Nmin], [min(LCDM_samples.T[i].flatten()), max(LCDM_samples.T[i].flatten())], c='k', lw=1.5)
    ax[i].text(Nmin, min(LCDM_samples.T[i].flatten()), r'$n_\mathrm{min}$', rotation=90, ha='right')
    ax[i].set_ylabel(labs[i])
    ax[i].set_xlabel('iteration')
    
    
plt.tight_layout()
#plt.savefig('plots/convergence_LCDM.png')
plt.show()

## Plot the 1$\sigma$ and 2$\sigma$ contours

In [ ]:
LCDM_samples = LCDM_samples[Nmin:, :].reshape((-1, ndim))


meanLCDM = np.mean(LCDM_samples, axis=0)
stdLCDM = np.var(LCDM_samples, axis=0)
maxLCDM=[]
for i in range(len(meanLCDM)):
    like = np.histogram(LCDM_samples.T[i], bins=1000)
    i_max=np.argmax(like[0])
    max_val = (like[1][i_max]+like[1][i_max+1])/2
    maxLCDM.append(max_val)

# LCDM_samples[:,0] = LCDM_samples[:,0] * LCDM_samples[:,2]**2 / 100**2


fig = corner(LCDM_samples[:,:3], quantiles=(.16,.84),  levels=(1-np.exp(-0.5),1-np.exp(-0.5*4)),# range=[(0.28,0.35),(0.01,0.08), (64,70)],
             labels=labs, smooth=True, smooth1d=True, bins=20, plot_datapoints=False, fill_contours=True, contourf_kwargs=dict(colors=None, cmap='Greens'))#,
#              truths=maxLCDM)




plt.tight_layout()

#plt.savefig('plots/LCDM' + data_sets_str[:-1] + '.pdf')

## Visualize the results

In [ ]:
v = np.percentile(LCDM_samples,[16, 50, 84], axis = 0)
v = np.asarray([v[1], v[2]-v[1],v[1]-v[0]]).T

omegam, stdmp, stdmm = v[0]

omegab, stdbp, stdbm = v[1]
H0, stdhp, stdhm = v[2]
    
    
#model
z = SNdata.get_data().T[0]
best_fit_cosmo_LCDM = cosmology(omegam=maxLCDM[0],omegac=1-maxLCDM[0], omegab = maxLCDM[1], Hzero=maxLCDM[2], z_num = z_num, rd_num=rd_num)
SNdata.set_param(v.T[0][-6:-2])
Qdata.set_param(v.T[0][-2:])

In [ ]:
bic_lcdm = BIC_LCDM(v.T[0],data_sets)

print('BIC(LCDM) = ' + str(bic_lcdm))

In [ ]:
fig = plt.figure(figsize=(8,8))
gs = gridspec.GridSpec(3, 1)
gs.update(hspace=0)


ax1 = plt.subplot(gs[0:2, 0])

if 'BAO'  in data_set_names or 'CMB' in data_set_names:    
    ax1.set_title(r'$\Omega_m = {%1.3f}^{+%1.3f}_{-%1.3f},\, H_0 = {%1.1f}^{+%1.1f}_{-%1.1f}\,\frac{\mathrm{km}/s}{\mathrm{Mpc}}$' % (omegam, stdmp,stdmm, H0, stdhm, stdhp))
else:
    ax1.set_title(r'$\Omega_m = {%1.3f}^{+%1.3f}_{-%1.3f}$' % (omegam, stdmp,stdmm))
    
if 'Quasars' in data_set_names:
    ax1.errorbar(Qdata.distance_modulus().T[0], Qdata.distance_modulus().T[1], yerr=Qdata.delta_distance_modulus(), linestyle='none', marker='o', color='orange', ecolor='yellow', markersize=3, alpha=0.6, zorder=0, label=r'quasars')
if 'SN' in data_set_names:
    ax1.errorbar(SNdata.distance_modulus().T[0], SNdata.distance_modulus().T[1], yerr=SNdata.delta_distance_modulus(), linestyle='none', marker='o', color='cyan', ecolor='blue', markersize=3, alpha=0.6, zorder=1, label=r'SNe')
if 'BAO' in data_set_names:
    ax1.errorbar(BAOdata.distance_modulus(best_fit_cosmo_LCDM).T[0], BAOdata.distance_modulus(best_fit_cosmo_LCDM).T[1], yerr=BAOdata.delta_distance_modulus(best_fit_cosmo_LCDM), linestyle='none', marker='o', color='red', ecolor='black', markersize=3, alpha=0.6, zorder=1, label=r'BAO')



zPlot = np.logspace(-3,np.log10(6),100)
ax1.plot(zPlot, best_fit_cosmo_LCDM.distance_modulus(zPlot), c='k', label=r'best fit')
ax1.fill_between(zPlot, cosmology(omegam+stdmp,1-omegam-stdmp, omegab=omegab+stdbp, Hzero=H0+stdhp).distance_modulus(zPlot), cosmology(omegam-stdmm,1-omegam+stdmm, omegab=omegab-stdbm, Hzero=H0-stdhm).distance_modulus(zPlot), color='gray', alpha=0.3)
ax1.plot(zPlot, cosmology(Omega_m_preset, Omega_c_preset, Hzero=100*h).distance_modulus(zPlot), ls = '--', c='gray', label=r'$\Lambda$CDM')

ax1.set_xlim(0.01,6)
ax1.set_ylim(32,55)


ax1.set_ylabel(r'distance modulus $\mu$')
ax1.set_xticklabels([])

ax1.grid('--', dashes=(5,5))
ax1.legend()


ax2 = plt.subplot(gs[2, 0])

ax2.plot(zPlot, best_fit_cosmo_LCDM.distance_modulus(zPlot) - cosmology(Omega_m_preset, Omega_c_preset).distance_modulus(zPlot), 'k')

# ax2.set_xscale('log')
ax2.set_xlim(0.01,6)

ax2.text(0.97, .06, r'$\mathrm{BIC}(\Lambda\mathrm{CDM}) = %1.0f$' % (BIC_LCDM(v.T[0],data_sets)), ha='right', va='bottom', transform=ax2.transAxes, bbox=dict(facecolor=(1,1,1,.8), edgecolor='lightgray', boxstyle='round'))


ax2.grid('--', dashes=(5,5))
ax2.set_xlabel(r'$z$')
ax2.set_ylabel(r'$\mu_\mathrm{fit} - \mu_{\Lambda\mathrm{CDM}}$')

plt.tight_layout()
plt.savefig('plots/Hubble_LCDM' + data_sets_str[:-1] + '.pdf')
plt.show()

## Joint fit of all data sets

### Here, we load and analyse all chains separately. Results are a list of best fit parameters and a full posterior plot

In [ ]:
LCDM_chains = {}
LCDM_chains['SN'] = HDFBackend('chains/LCDM_SN_512x1000.h5')
LCDM_chains['SN_Q'] = HDFBackend('chains/LCDM_SN_Quasars_512x1000.h5')
LCDM_chains['SN_Q_BAO'] = HDFBackend('chains/LCDM_SN_Quasars_BAO_512x1000.h5')
LCDM_chains['SN_BAO_CMB'] = HDFBackend('chains/LCDM_SN_CMB_512x1000.h5')
LCDM_chains['SN_Q_BAO_CMB'] = HDFBackend('chains/LCDM_SN_Quasars_BAO_CMB_512x1000.h5')
LCDM_chains['BAO'] = HDFBackend('chains/LCDM_BAO_512x1000.h5')
LCDM_chains['CMB'] = HDFBackend('chains/LCDM_CMB_512x1000.h5')
LCDM_chains['BAO_CMB'] = HDFBackend('chains/LCDM_BAO_CMB_512x1000.h5')

In [ ]:
data_set_combinations = ['SN','SN_Q','SN_Q_BAO','SN_BAO_CMB','SN_Q_BAO_CMB','BAO','CMB','BAO_CMB']
data_set_combin_obs = {}
data_set_combin_obs['SN'] = [SNdata]
data_set_combin_obs['SN_Q'] = [SNdata,Qdata]
data_set_combin_obs['SN_Q_BAO'] = [SNdata,Qdata,BAOdata]
data_set_combin_obs['SN_BAO_CMB'] = [SNdata,BAOdata,CMBdata]
data_set_combin_obs['SN_Q_BAO_CMB'] = [SNdata,Qdata,BAOdata,CMBdata]
data_set_combin_obs['BAO'] = [BAOdata]
data_set_combin_obs['CMB'] = [CMBdata]
data_set_combin_obs['BAO_CMB'] = [BAOdata,CMBdata]

In [ ]:
# Generate a list of best fit paramters and calculate the BIC for LCDM

bic_lcdmlist = {}

for set_name in data_set_combinations:
    sample = LCDM_chains[set_name].get_chain()
    Nmin = 300
    while Nmin < nsteps and np.all(LCDM_chains[set_name].get_autocorr_time(tol=0,discard=Nmin)/(nsteps)*30 > 1):
        Nmin+=100
    dellist = np.unique(np.where(np.isclose(sample[-1] - sample[0], 0))[0])
    sample = np.delete(sample, np.unique(dellist), axis=1)[Nmin:, :].reshape((-1, ndim))
    
    data_sets = data_set_combin_obs[set_name]
    
    vtemp = np.percentile(sample,[16, 50, 84], axis = 0)
    vtemp = np.asarray([vtemp[1], vtemp[2]-vtemp[1],vtemp[1]-vtemp[0]]).T
    
    bic_lcdmlist[set_name] = BIC_LCDM(vtemp.T[0], data_set_combin_obs[set_name])
    
    print(set_name + ':')
    print('   Om_m:' + str(np.round(vtemp[0],4)))
    print('   Om_b:' + str(np.round(vtemp[1],4)*100))
    print('   H0:  ' + str(np.round(vtemp[2],2)))
    print('   BIC: ' + str(np.round(bic_lcdmlist[set_name])))

In [ ]:
Nmin = 300
for sampler in LCDM_chains.values():
    nsteps, nwalkers, ndim = sampler.get_chain().shape
    while Nmin < nsteps and np.all(sampler.get_autocorr_time(tol=0,discard=Nmin)/(nsteps)*30 > 1):
        Nmin+=100
print('Nmin=',Nmin)

In [ ]:
ranges_LCDM_min=[0, 0, 60., -5, -10, -30, -.5, 0, 0]
ranges_LCDM_max=[1, 0.1, 80., 5, 10, -10, .5, 10, 3]

In [ ]:
# Plot the posteriors

samples = LCDM_chains['SN'].get_chain()[Nmin:,np.delete(range(nwalkers), calcdellist(LCDM_chains['SN'])),:].reshape((-1, LCDM_chains['SN'].get_chain().shape[-1]))
fig=corner(samples[:,:3], levels=(1-np.exp(-0.5),1-np.exp(-0.5*4)),
            labels=labs, smooth=True, smooth1d=True, bins=25, plot_datapoints=False, color='green',
            fill_contours=True, contourf_kwargs=dict(colors=['white', 'lightgreen', 'green']),
            range=[(0.15,.5), (0.03,0.07), (62,75)])

samples = LCDM_chains['SN_Q'].get_chain()[Nmin:,np.delete(range(nwalkers), calcdellist(LCDM_chains['SN_Q'])),:].reshape((-1, LCDM_chains['SN_Q'].get_chain().shape[-1]))
corner(samples[:,:3], levels=(1-np.exp(-0.5),1-np.exp(-0.5*4)),
            labels=labs, smooth=True, smooth1d=True, bins=25, plot_datapoints=False, color='blue', 
            fill_contours=True, contourf_kwargs=dict(colors=['white', 'lightblue', 'blue'], alpha=0.6),
            fig=fig, range=[(0.15,.5), (0.03,0.07), (62,75)])

samples = LCDM_chains['BAO'].get_chain()[Nmin:,np.delete(range(nwalkers), calcdellist(LCDM_chains['BAO'])),:].reshape((-1, LCDM_chains['BAO'].get_chain().shape[-1]))
fig=corner(samples[:,:3], levels=(1-np.exp(-0.5*1),1-np.exp(-0.5*4)),
            labels=labs, smooth=True, smooth1d=True, bins=25, plot_datapoints=False, color='red', 
            fill_contours=True, contourf_kwargs=dict(colors=['white', 'pink', 'red'], alpha=0.4),
            fig=fig, range=[(0.15,.5), (0.03,0.07), (62,75)])

samples = LCDM_chains['CMB'].get_chain()[Nmin:,np.delete(range(nwalkers), calcdellist(LCDM_chains['CMB'])),:].reshape((-1, LCDM_chains['CMB'].get_chain().shape[-1]))
corner(samples[:,:3], levels=(1-np.exp( -0.5*1),1-np.exp(-0.5*4)),
            labels=labs, smooth=True, smooth1d=True, bins=25, plot_datapoints=False, color='orange', 
            fill_contours=True, contourf_kwargs=dict(colors=['white', 'yellow', 'orange'], alpha=0.4),
            fig=fig, range=[(0.15,.5), (0.03,0.07), (62,75)])

#samples = LCDM_chains['SN_Q_BAO_CMB'].get_chain()[Nmin:,np.delete(range(nwalkers), calcdellist(LCDM_chains['SN_Q_BAO_CMB'])),:].reshape((-1, LCDM_chains['SN_Q_BAO_CMB'].get_chain().shape[-1]))
#corner(samples[:,:3], levels=(1-np.exp(-0.5*1),1-np.exp(-0.5*4)),
#            labels=labs, smooth=True, smooth1d=True, bins=25, plot_datapoints=False, color='cyan', 
#            fill_contours=True, contourf_kwargs=dict(colors=['white', 'lightblue', 'cyan'], alpha=0.4),
#            fig=fig, range=[(0.15,.5), (0.03,0.07), (62,75)])


# samples = LCDM_chains['SN_BAO_CMB'].get_chain()[Nmin:,:].reshape((-1, LCDM_chains['SN_Q_BAO_CMB'].get_chain().shape[-1]))
# corner(samples[:,:3], levels=(1-np.exp(-0.5*1),1-np.exp(-0.5*4)),
#             labels=labs, smooth=True, smooth1d=True, bins=25, plot_datapoints=False, color='k', 
#             fill_contours=True, contourf_kwargs=dict(colors=['white', 'lightgray', 'gray'], alpha=0.4),
#             fig=fig, range=[(0.15,.5), (0.03,0.07), (62,75)])

plt.tight_layout()
#plt.savefig('plots/posterior_LCDM.pdf')


plt.show()

In [ ]:
range_plot = [(0.2,.4), (0.04,0.06), (65,70)]

samples = LCDM_chains['BAO'].get_chain()[Nmin:,np.delete(range(nwalkers), calcdellist(LCDM_chains['BAO'])),:].reshape((-1, LCDM_chains['BAO'].get_chain().shape[-1]))
fig=corner(samples[:,:3], levels=(1-np.exp(-0.5*1),1-np.exp(-0.5*4)),
            labels=labs, smooth=True, smooth1d=True, bins=25, plot_datapoints=False, color='red', 
            fill_contours=True, contourf_kwargs=dict(colors=['white', 'pink', 'red'], alpha=0.4),
            range=range_plot)

samples = LCDM_chains['CMB'].get_chain()[Nmin:,np.delete(range(nwalkers), calcdellist(LCDM_chains['CMB'])),:].reshape((-1, LCDM_chains['CMB'].get_chain().shape[-1]))
corner(samples[:,:3], levels=(1-np.exp( -0.5*1),1-np.exp(-0.5*4)),
            labels=labs, smooth=True, smooth1d=True, bins=25, plot_datapoints=False, color='orange', 
            fill_contours=True, contourf_kwargs=dict(colors=['white', 'yellow', 'orange'], alpha=0.4),
            fig=fig, range=range_plot)

samples = LCDM_chains['BAO_CMB'].get_chain()[Nmin:,np.delete(range(nwalkers), calcdellist(LCDM_chains['BAO_CMB'])),:].reshape((-1, LCDM_chains['BAO_CMB'].get_chain().shape[-1]))
corner(samples[:,:3], levels=(1-np.exp(-0.5*1),1-np.exp(-0.5*4)),
            labels=labs, smooth=True, smooth1d=True, bins=25, plot_datapoints=False, color='cyan', 
            fill_contours=True, contourf_kwargs=dict(colors=['white', 'lightblue', 'cyan'], alpha=0.4),
            fig=fig, range=range_plot)



plt.tight_layout()
plt.savefig('plots/posterior_LCDM.pdf')


plt.show()

## Full posterior distribution

In [ ]:
# For LCDM, we also generate the full posterior distribution plot, with marginalised parameters:

In [ ]:
samples = LCDM_chains['SN'].get_chain()[Nmin:,np.delete(range(nwalkers), calcdellist(LCDM_chains['SN'])),:].reshape((-1, LCDM_chains['SN'].get_chain().shape[-1]))
fig=corner(samples, levels=(1-np.exp(-0.5),1-np.exp(-0.5*4)),
            labels=labs, smooth=True, smooth1d=True, bins=25, plot_datapoints=False, color='green',
            fill_contours=True, contourf_kwargs=dict(colors=['white', 'lightgreen', 'green']),
            range=[(0.15,.5), (0.03,0.07), (62,75), (0,0.3), (0,5), (-22,-18), (-1,1) ,(5,10), (1,2)])

samples = LCDM_chains['SN_Q'].get_chain()[Nmin:,np.delete(range(nwalkers), calcdellist(LCDM_chains['SN_Q'])),:].reshape((-1, LCDM_chains['SN_Q'].get_chain().shape[-1]))
corner(samples, levels=(1-np.exp(-0.5),1-np.exp(-0.5*4)),
            labels=labs, smooth=True, smooth1d=True, bins=25, plot_datapoints=False, color='blue', 
            fill_contours=True, contourf_kwargs=dict(colors=['white', 'lightblue', 'blue'], alpha=0.6),
            fig=fig, range=[(0.15,.5), (0.03,0.07), (62,75), (0,0.3), (0,5), (-22,-18), (-1,1) ,(5,10), (1,2)])

samples = LCDM_chains['SN_Q_BAO_CMB'].get_chain()[Nmin:,np.delete(range(nwalkers), calcdellist(LCDM_chains['SN_Q_BAO_CMB'])),:].reshape((-1, LCDM_chains['SN_Q_BAO_CMB'].get_chain().shape[-1]))
corner(samples, levels=(1-np.exp(-0.5*1),1-np.exp(-0.5*4)),
            labels=labs, smooth=True, smooth1d=True, bins=25, plot_datapoints=False, color='cyan', 
            fill_contours=True, contourf_kwargs=dict(colors=['white', 'lightblue', 'cyan'], alpha=0.4),
            fig=fig, range=[(0.15,.5), (0.03,0.07), (62,75), (0,0.3), (0,5), (-22,-18), (-1,1) ,(5,10), (1,2)])


#samples = LCDM_chains['SN_BAO_CMB'].get_chain()[Nmin:,:].reshape((-1, LCDM_chains['SN_Q_BAO_CMB'].get_chain().shape[-1]))
#corner(samples[:,:3], levels=(1-np.exp(-0.5*1),1-np.exp(-0.5*4)),
#             labels=labs, smooth=True, smooth1d=True, bins=25, plot_datapoints=False, color='k', 
#             fill_contours=True, contourf_kwargs=dict(colors=['white', 'lightgray', 'gray'], alpha=0.4),
#             fig=fig, range=[(0.15,.5), (0.03,0.07), (62,75)])

#plt.tight_layout()
#plt.tick_params(axis='both', which='major', labelsize=10)
#plt.tick_params(axis='both', which='minor', labelsize=8)


plt.savefig('plots/posterior_LCDM_full.pdf')


plt.show()

# Model 1: o$\Lambda$LCDM

## which data sets to include?

In [ ]:
model = 'oLCDM'
CMBdata = CMB_data(model, 'Planck18')

In [ ]:
data_sets = [SNdata,Qdata,BAOdata,CMBdata]
data_set_names = [set.name for set in data_sets]
data_sets_str = '_' + '_'.join([name for name in data_set_names]) + '_' 

## Define the likelihood and priors

In [ ]:
ranges_oLCDM_min=[0, 0, 0, 60., -5, -10, -30, -.5, 0, 0]
ranges_oLCDM_max=[1, 1.5, .1, 80., 5, 10, -10, .5, 10, 3]
def lnprob_oLCDM(theta):
    l = likelihood(theta, data_sets, ranges_oLCDM_min, ranges_oLCDM_max, model = 'oLCDM', rd_num = rd_num, z_num = z_num)
    return l.logprobability_gauss_prior()

## define a reference BIC

In [ ]:
theta_oLCDM = [Omega_m_preset, Omega_c_preset, omega_baryon_preset, 67.27, a, b, MB, -0.05, beta_prime, s]

BIC_oLCDM = lambda theta, data_sets_var: np.log(sum([len(d.get_data()) for d in data_sets_var])) * len(theta) -2*lnprob_oLCDM(theta)

## run the MCMC sampler  [arXiv:1202.3665]...

In [ ]:
ndim, nwalkers, nsteps = 10, 512, 1000
pos0 = np.random.uniform(ranges_oLCDM_min, ranges_oLCDM_max, (nwalkers,len(ranges_oLCDM_max)))

pool = Pool(cpu_count())
write = HDFBackend('chains/testing/150120_oLCDM_v2_znum_TCMB_anard' + data_sets_str + str(nwalkers) + 'x' + str(nsteps) + '.h5')
sampler = EnsembleSampler(nwalkers, ndim, lnprob_oLCDM, pool=pool)#, backend=write)

sampler.run_mcmc(pos0, nsteps, progress=True)

## ...or load existing chains

In [ ]:
oLCDM_sampler = HDFBackend('chains/oLCDM_CMB_512x1000.h5', read_only=True)
#oLCDM_sampler = sampler

nsteps, nwalkers, ndim = oLCDM_sampler.get_chain().shape

labs = [r'$\Omega_m$', r'$\Omega_\Lambda$', r'$\Omega_b$', r'$H_0\ [\frac{\mathrm{km}/s}{\mathrm{Mpc}}]$', r'$\alpha$', r'$\beta$', r'$M_B$', r'$\Delta M_B$', r'$\beta^\prime$', r'$\delta$']

## Check and visualise the convergence: $\tau_f / n_\mathrm{steps} < 1/30$

In [ ]:
Nmin = 300
while Nmin < nsteps and np.all(oLCDM_sampler.get_autocorr_time(tol=0,discard=Nmin)/(nsteps)*30 > 1):
    Nmin+=100
    
print('Nmin = {}'.format(Nmin))

In [ ]:
oLCDM_samples = oLCDM_sampler.get_chain()

dellist = np.unique(np.where(np.isclose(oLCDM_samples[-1] - oLCDM_samples[0], 0))[0])
oLCDM_samples = np.delete(oLCDM_samples, np.unique(dellist), axis=1)

f, ax = plt.subplots(len(oLCDM_samples.T), 1, figsize=(6,3*len(oLCDM_samples.T)))

for i in range(len(oLCDM_samples.T)):
    ax[i].plot(oLCDM_samples.T[i].T, lw=0.5)
    ax[i].plot([Nmin, Nmin], [min(oLCDM_samples.T[i].flatten()), max(oLCDM_samples.T[i].flatten())], c='k', lw=1.5)
    ax[i].text(Nmin, min(oLCDM_samples.T[i].flatten()), r'$n_\mathrm{min}$', rotation=90, ha='right')
    ax[i].set_ylabel(labs[i])
    ax[i].set_xlabel('iteration')
    
    
plt.tight_layout()
#plt.savefig('plots/convergence_oLCDM_new.png')
plt.show()

## Plot the 1$\sigma$ and 2$\sigma$ contours

In [ ]:
oLCDM_samples = oLCDM_samples[Nmin:].reshape((-1, ndim))
#oLCDM_samples = oLCDM_samples.reshape((-1, ndim))


meanoLCDM = np.mean(oLCDM_samples, axis=0)
stdoLCDM = np.var(oLCDM_samples, axis=0)
maxoLCDM=[]
for i in range(len(meanoLCDM)):
    like = np.histogram(oLCDM_samples.T[i], bins=500)
    i_max=np.argmax(like[0])
    max_val = (like[1][i_max]+like[1][i_max+1])/2
    maxoLCDM.append(max_val)

fig = corner(oLCDM_samples[:,:4], quantiles=(.16,.84),  levels=(1-np.exp(-0.5),1-np.exp(-0.5*4)), range=[(0,0.8),(0,1),(0,.1), (60,80)],
             labels=labs, smooth=True, smooth1d=True, bins=25, plot_datapoints=False, fill_contours=True, contourf_kwargs=dict(colors=None, cmap='Greens'))#,
#              truths=maxoLCDM)




plt.tight_layout()
#plt.savefig('plots/testing/110120_LCDM_v2_1089_TCMB_anard' + data_sets_str[:-1] + '.pdf')

## Visualize the results

In [ ]:
v = np.percentile(oLCDM_samples,[16, 50, 84], axis = 0)
v = np.asarray([v[1], v[2]-v[1],v[1]-v[0]]).T

omegam, stdmp, stdmm = v[0]
omegac, stdcp, stdcm = v[1]

omegab, stdbp, stdbm = v[2]
H0, stdhp, stdhm = v[3]
    
    
#model
z = SNdata.get_data().T[0]
best_fit_cosmo_oLCDM = cosmology(*maxoLCDM[:2], omegab = maxoLCDM[2],  omegag=omega_gamma_preset, Hzero=maxoLCDM[3])
SNdata.set_param(v.T[0][4:8])
Qdata.set_param(v.T[0][8:])

In [ ]:
fig = plt.figure(figsize=(8,8))
gs = gridspec.GridSpec(3, 1)
gs.update(hspace=0)


ax1 = plt.subplot(gs[0:2, 0])

if 'BAO'  in data_set_names or 'CMB' in data_set_names:    
    ax1.set_title(r'$\Omega_m = {%1.2f}^{+%1.2f}_{-%1.2f},\, \Omega_\Lambda = {%1.2f}^{+%1.2f}_{-%1.2f},\, H_0 = {%1.1f}^{+%1.1f}_{-%1.1f} \,\frac{\mathrm{km}/s}{\mathrm{Mpc}}$' % (omegam, stdmp,stdmm, omegac, stdcp, stdcm, H0, stdhm, stdhp))
else:
    ax1.set_title(r'$\Omega_m = {%1.2f}^{+%1.2f}_{-%1.2f},\, \Omega_\Lambda = {%1.2f}^{+%1.2f}_{-%1.2f}$' % (omegam, stdmp,stdmm, omegac, stdcp, stdcm))

if 'Quasars' in data_set_names:
    ax1.errorbar(Qdata.distance_modulus().T[0], Qdata.distance_modulus().T[1], yerr=Qdata.delta_distance_modulus(), linestyle='none', marker='o', color='orange', ecolor='yellow', markersize=3, alpha=0.6, zorder=0, label=r'quasars')
if 'SN' in data_set_names:
    ax1.errorbar(SNdata.distance_modulus().T[0], SNdata.distance_modulus().T[1], yerr=SNdata.delta_distance_modulus(), linestyle='none', marker='o', color='cyan', ecolor='blue', markersize=3, alpha=0.6, zorder=1, label=r'SNe')
if 'BAO' in data_set_names:
    ax1.errorbar(BAOdata.distance_modulus(best_fit_cosmo_oLCDM).T[0], BAOdata.distance_modulus(best_fit_cosmo_oLCDM).T[1], yerr=BAOdata.delta_distance_modulus(best_fit_cosmo_oLCDM), linestyle='none', marker='o', color='red', ecolor='black', markersize=3, alpha=0.6, zorder=1, label=r'BAO')



zPlot = np.logspace(-3,np.log10(6),100)
ax1.plot(zPlot, best_fit_cosmo_oLCDM.distance_modulus(zPlot), c='k', label=r'best fit')
ax1.fill_between(zPlot, cosmology(omegam+stdmp,omegac+stdcp, omegab=omegab+stdbp, Hzero=H0+stdhp).distance_modulus(zPlot), cosmology(omegam-stdmm,omegac-stdcm, omegab=omegab-stdbm, Hzero=H0-stdhm).distance_modulus(zPlot), color='gray', alpha=0.3)
ax1.plot(zPlot, cosmology(Omega_m_preset, Omega_c_preset).distance_modulus(zPlot), ls = '--', c='gray', label=r'$\Lambda$CDM')

ax1.set_xlim(0.01,6)
ax1.set_ylim(32,55)


ax1.set_ylabel(r'distance modulus $\mu$')
ax1.set_xticklabels([])

ax1.grid('--', dashes=(5,5))
ax1.legend()


ax2 = plt.subplot(gs[2, 0])

ax2.plot(zPlot, best_fit_cosmo_oLCDM.distance_modulus(zPlot) - cosmology(Omega_m_preset, Omega_c_preset).distance_modulus(zPlot), 'k')

# ax2.set_xscale('log')
ax2.set_xlim(0.01,6)

ax2.text(0.97, .06, r'$\Delta \mathrm{BIC}(o\Lambda\mathrm{CDM}) = %1.0f$' % (BIC_oLCDM(v.T[0],data_sets) - bic_lcdm), ha='right', va='bottom', transform=ax2.transAxes, bbox=dict(facecolor=(1,1,1,.8), edgecolor='lightgray', boxstyle='round'))


ax2.grid('--', dashes=(5,5))
ax2.set_xlabel(r'$z$')
ax2.set_ylabel(r'$\mu_\mathrm{fit} - \mu_{\Lambda\mathrm{CDM}}$')

plt.tight_layout()
#plt.savefig('plots/Hubble_oLCDM_test' + data_sets_str[:-1] + '.pdf')
plt.show()

## Joint fit of all data sets

In [ ]:
oLCDM_chains = {}
oLCDM_chains['SN'] = HDFBackend('chains/oLCDM_SN_512x1000.h5')
oLCDM_chains['SN_Q'] = HDFBackend('chains/oLCDM_SN_Quasars_512x1000.h5')
oLCDM_chains['SN_Q_BAO'] = HDFBackend('chains/oLCDM_SN_Quasars_BAO_512x1000.h5')
oLCDM_chains['SN_BAO_CMB'] = HDFBackend('chains/oLCDM_SN_BAO_CMB_512x1000.h5')
oLCDM_chains['SN_Q_BAO_CMB'] = HDFBackend('chains/oLCDM_SN_Quasars_BAO_CMB_512x1000.h5')
oLCDM_chains['BAO'] = HDFBackend('chains/oLCDM_BAO_512x1000.h5')
oLCDM_chains['CMB'] = HDFBackend('chains/oLCDM_CMB_512x1000.h5')
oLCDM_chains['BAO_CMB'] = HDFBackend('chains/oLCDM_BAO_CMB_512x1000.h5')

In [ ]:
#oLCDM_chains = {}

#oLCDM_chains['BAO'] = HDFBackend('chains/testing/150120_oLCDM_v2_znum_TCMB_anard_BAO_100x500.h5')
#oLCDM_chains['CMB'] = HDFBackend('chains/testing/150120_oLCDM_v2_znum_TCMB_anard_CMB_100x500.h5')
#oLCDM_chains['BAO_CMB'] = HDFBackend('chains/testing/150120_oLCDM_v2_znum_TCMB_anard_BAO_CMB_100x500.h5')

#oLCDM_chains['BAO'] = HDFBackend('chains/oLCDM_BAO_512x1000.h5')
#oLCDM_chains['CMB'] = HDFBackend('chains/oLCDM_CMB_512x1000.h5')
#oLCDM_chains['BAO_CMB'] = HDFBackend('chains/oLCDM_BAO_CMB_512x1000.h5')

In [ ]:
data_set_combinations = ['BAO_CMB','BAO','CMB','SN','SN_Q','SN_Q_BAO','SN_BAO_CMB','SN_Q_BAO_CMB']
data_set_combin_obs = {}
data_set_combin_obs['BAO_CMB'] = [BAOdata,CMBdata]
data_set_combin_obs['SN'] = [SNdata]
data_set_combin_obs['SN_Q'] = [SNdata,Qdata]
data_set_combin_obs['SN_Q_BAO'] = [SNdata,Qdata,BAOdata]
data_set_combin_obs['SN_BAO_CMB'] = [SNdata,BAOdata,CMBdata]
data_set_combin_obs['SN_Q_BAO_CMB'] = [SNdata,Qdata,BAOdata,CMBdata]
data_set_combin_obs['BAO'] = [BAOdata]
data_set_combin_obs['CMB'] = [CMBdata]

In [ ]:
# Generate a list of best fit paramters and calculate the Delta BIC of oLCDM with respect to LCDM

for set_name in data_set_combinations:
    sample = oLCDM_chains[set_name].get_chain()
    Nmin = 300
    while Nmin < nsteps and np.all(oLCDM_chains[set_name].get_autocorr_time(tol=0,discard=Nmin)/(nsteps)*30 > 1):
        Nmin+=100
    dellist = np.unique(np.where(np.isclose(sample[-1] - sample[0], 0))[0])
    sample = np.delete(sample, np.unique(dellist), axis=1)[Nmin:, :].reshape((-1, ndim))
    
    data_sets = data_set_combin_obs[set_name]
    
    vtemp = np.percentile(sample,[16, 50, 84], axis = 0)
    vtemp = np.asarray([vtemp[1], vtemp[2]-vtemp[1],vtemp[1]-vtemp[0]]).T
    print(set_name + ':')
    print('   Om_m:' + str(np.round(vtemp[0],4)))
    print('   Om_c:' + str(np.round(vtemp[1],4)))
    print('   Om_b:' + str(np.round(vtemp[2],5)*100))
    print('   H0:  ' + str(np.round(vtemp[3],2)))
    print('   dBIC:' + str(np.round(BIC_oLCDM(vtemp.T[0], data_set_combin_obs[set_name])-bic_lcdmlist[set_name])))

In [ ]:
Nmin = 300
for sampler in oLCDM_chains.values():
    nsteps, nwalkers, ndim = sampler.get_chain().shape
    while Nmin < nsteps and np.all(sampler.get_autocorr_time(tol=0,discard=Nmin)/(nsteps)*30 > 1):
        Nmin+=100
print('Nmin=',Nmin)

In [ ]:
samples = oLCDM_chains['SN'].get_chain()[Nmin:,np.delete(range(nwalkers), calcdellist(oLCDM_chains['SN'])),:].reshape((-1, oLCDM_chains['SN'].get_chain().shape[-1]))
fig=corner(samples[:,:4], levels=(1-np.exp(-0.5),1-np.exp(-0.5*4)),
            labels=labs, smooth=True, smooth1d=True, bins=25, plot_datapoints=False, color='green',
            fill_contours=True, contourf_kwargs=dict(colors=['white', 'lightgreen', 'green']),
            range=[(0,.6), (0,1.25), (0.03,0.07), (62,75)])

samples = oLCDM_chains['SN_Q'].get_chain()[Nmin:,np.delete(range(nwalkers), calcdellist(oLCDM_chains['SN_Q'])),:].reshape((-1, oLCDM_chains['SN_Q'].get_chain().shape[-1]))
corner(samples[:,:4], levels=(1-np.exp(-0.5),1-np.exp(-0.5*4)),
            labels=labs, smooth=True, smooth1d=True, bins=25, plot_datapoints=False, color='blue', 
            fill_contours=True, contourf_kwargs=dict(colors=['white', 'lightblue', 'blue'], alpha=0.6),
            fig=fig, range=[(0,.6), (0,1.25), (0.03,0.07), (62,75)])

samples = oLCDM_chains['BAO'].get_chain()[Nmin:,np.delete(range(nwalkers), calcdellist(oLCDM_chains['BAO'])),:].reshape((-1, oLCDM_chains['BAO'].get_chain().shape[-1]))
corner(samples[:,:4], levels=(1-np.exp(-0.5*1),1-np.exp(-0.5*4)),
            labels=labs, smooth=True, smooth1d=True, bins=25, plot_datapoints=False, color='red', 
            fill_contours=True, contourf_kwargs=dict(colors=['white', 'pink', 'red'], alpha=0.4),
            fig=fig, range=[(0,.6), (0,1.25), (0.03,0.07), (62,75)])

samples = oLCDM_chains['CMB'].get_chain()[Nmin:,np.delete(range(nwalkers), calcdellist(oLCDM_chains['CMB'])),:].reshape((-1, oLCDM_chains['CMB'].get_chain().shape[-1]))
corner(samples[:,:4], levels=(1-np.exp(-0.5*1),1-np.exp(-0.5*4)),
            labels=labs, smooth=True, smooth1d=True, bins=25, plot_datapoints=False, color='orange', 
            fill_contours=True, contourf_kwargs=dict(colors=['white', 'yellow', 'orange'], alpha=0.4),
            fig=fig, range=[(0,.6), (0,1.25), (0.03,0.07), (62,75)])

samples = oLCDM_chains['SN_Q_BAO_CMB'].get_chain()[Nmin:,np.delete(range(nwalkers), calcdellist(oLCDM_chains['SN_Q_BAO_CMB'])),:].reshape((-1, oLCDM_chains['SN_Q_BAO_CMB'].get_chain().shape[-1]))
corner(samples[:,:4], levels=(1-np.exp(-0.5*1),1-np.exp(-0.5*4)),
            labels=labs, smooth=True, smooth1d=True, bins=25, plot_datapoints=False, color='cyan', 
            fill_contours=True, contourf_kwargs=dict(colors=['white', 'lightblue', 'cyan'], alpha=0.4),
            fig=fig, range=[(0,.6), (0,1.25), (0.03,0.07), (62,75)])


# samples = oLCDM_chains['SN_BAO_CMB'].get_chain()[Nmin:,:].reshape((-1, oLCDM_chains['SN_BAO_CMB'].get_chain().shape[-1]))
# corner(samples[:,:4], levels=(1-np.exp(-0.5*1),1-np.exp(-0.5*4)),
#             labels=labs, smooth=True, smooth1d=True, bins=25, plot_datapoints=False, color='k', 
#             fill_contours=True, contourf_kwargs=dict(colors=['white', 'lightgray', 'gray'], alpha=0.4),
#             fig=fig, range=[(0,.6), (0,1.25), (0.03,0.07), (62,75)])

plt.tight_layout()
#plt.savefig('plots/posterior_oLCDM.pdf')


plt.show()

In [ ]:
posterior_range = [(0,.6), (0,1.25), (0.03,0.07), (60,80)]

samples = oLCDM_chains['BAO'].get_chain()[Nmin:,np.delete(range(nwalkers), calcdellist(oLCDM_chains['BAO'])),:].reshape((-1, oLCDM_chains['BAO'].get_chain().shape[-1]))
fig=corner(samples[:,:4], levels=(1-np.exp(-0.5*1),1-np.exp(-0.5*4)),
            labels=labs, smooth=True, smooth1d=True, bins=25, plot_datapoints=False, color='red', 
            fill_contours=True, contourf_kwargs=dict(colors=['white', 'pink', 'red'], alpha=0.4),
            range=posterior_range)

samples = oLCDM_chains['CMB'].get_chain()[Nmin:,np.delete(range(nwalkers), calcdellist(oLCDM_chains['CMB'])),:].reshape((-1, oLCDM_chains['CMB'].get_chain().shape[-1]))
corner(samples[:,:4], levels=(1-np.exp(-0.5*1),1-np.exp(-0.5*4)),
            labels=labs, smooth=True, smooth1d=True, bins=25, plot_datapoints=False, color='orange', 
            fill_contours=True, contourf_kwargs=dict(colors=['white', 'yellow', 'orange'], alpha=0.4),
            fig=fig, range=posterior_range)



samples = oLCDM_chains['BAO_CMB'].get_chain()[Nmin:,np.delete(range(nwalkers), calcdellist(oLCDM_chains['BAO_CMB'])),:].reshape((-1, oLCDM_chains['BAO_CMB'].get_chain().shape[-1]))
corner(samples[:,:4], levels=(1-np.exp(-0.5*1),1-np.exp(-0.5*4)),
            labels=labs, smooth=True, smooth1d=True, bins=25, plot_datapoints=False, color='cyan', 
            fill_contours=True, contourf_kwargs=dict(colors=['white', 'lightblue', 'cyan'], alpha=0.4),
            fig=fig, range=posterior_range)



#samples = oLCDM_chains['SN_Q_BAO_CMB'].get_chain()[Nmin:,:].reshape((-1, oLCDM_chains['SN_Q_BAO_CMB'].get_chain().shape[-1]))
#corner(samples[:,:4], levels=(1-np.exp(-0.5*1),1-np.exp(-0.5*4)),
#             labels=labs, smooth=True, smooth1d=True, bins=25, plot_datapoints=False, color='k', 
#             fill_contours=True, contourf_kwargs=dict(colors=['white', 'lightgray', 'gray'], alpha=0.4),
#             fig=fig, range=posterior_range)

plt.tight_layout()
plt.savefig('plots/posterior_oLCDM.pdf')


plt.show()

## Full posterior distribution

In [ ]:
#samples = oLCDM_chains['SN'].get_chain()[Nmin:,np.delete(range(nwalkers), calcdellist(oLCDM_chains['SN'])),:].reshape((-1, oLCDM_chains['SN'].get_chain().shape[-1]))
#fig=corner(samples, levels=(1-np.exp(-0.5),1-np.exp(-0.5*4)),
#            labels=labs, smooth=True, smooth1d=True, bins=25, plot_datapoints=False, color='green',
#            fill_contours=True, contourf_kwargs=dict(colors=['white', 'lightgreen', 'green']),
#            range=[(0.15,.5), (0.5,1.0), (0.03,0.07), (62,75), (0,0.3), (0,5), (-22,-18), (-1,1) ,(5,10), (1,2)])

samples = oLCDM_chains['SN_Q'].get_chain()[Nmin:,np.delete(range(nwalkers), calcdellist(oLCDM_chains['SN_Q'])),:].reshape((-1, oLCDM_chains['SN_Q'].get_chain().shape[-1]))
fig=corner(samples, levels=(1-np.exp(-0.5),1-np.exp(-0.5*4)),
            labels=labs, smooth=True, smooth1d=True, bins=25, plot_datapoints=False, color='blue', 
            fill_contours=True, contourf_kwargs=dict(colors=['white', 'lightblue', 'blue'], alpha=0.6),
            range=[(0.15,.5), (0.5,1.0), (0.03,0.07), (62,75), (0,0.3), (0,5), (-22,-18), (-1,1) ,(5,10), (1,2)])

samples = oLCDM_chains['BAO'].get_chain()[Nmin:,np.delete(range(nwalkers), calcdellist(oLCDM_chains['BAO'])),:].reshape((-1, oLCDM_chains['BAO'].get_chain().shape[-1]))
corner(samples, levels=(1-np.exp(-0.5*1),1-np.exp(-0.5*4)),
            labels=labs, smooth=True, smooth1d=True, bins=25, plot_datapoints=False, color='red', 
            fill_contours=True, contourf_kwargs=dict(colors=['white', 'pink', 'red'], alpha=0.4),
            fig=fig, range=[(0.15,.5), (0.5,1.0), (0.03,0.07), (62,75), (0,0.3), (0,5), (-22,-18), (-1,1) ,(5,10), (1,2)])

samples = oLCDM_chains['CMB'].get_chain()[Nmin:,np.delete(range(nwalkers), calcdellist(oLCDM_chains['CMB'])),:].reshape((-1, oLCDM_chains['CMB'].get_chain().shape[-1]))
corner(samples, levels=(1-np.exp(-0.5*1),1-np.exp(-0.5*4)),
            labels=labs, smooth=True, smooth1d=True, bins=25, plot_datapoints=False, color='yellow', 
            fill_contours=True, contourf_kwargs=dict(colors=['white', 'lightyellow', 'yellow'], alpha=0.4),
            fig=fig, range=[(0.15,.5), (0.5,1.0), (0.03,0.07), (62,75), (0,0.3), (0,5), (-22,-18), (-1,1) ,(5,10), (1,2)])


samples = oLCDM_chains['SN_Q_BAO_CMB'].get_chain()[Nmin:,np.delete(range(nwalkers), calcdellist(oLCDM_chains['SN_Q_BAO_CMB'])),:].reshape((-1, oLCDM_chains['SN_Q_BAO_CMB'].get_chain().shape[-1]))
corner(samples, levels=(1-np.exp(-0.5*1),1-np.exp(-0.5*4)),
            labels=labs, smooth=True, smooth1d=True, bins=25, plot_datapoints=False, color='cyan', 
            fill_contours=True, contourf_kwargs=dict(colors=['white', 'lightblue', 'cyan'], alpha=0.4),
            fig=fig, range=[(0.15,.5), (0.5,1.0), (0.03,0.07), (62,75), (0,0.3), (0,5), (-22,-18), (-1,1) ,(5,10), (1,2)])


#samples = LCDM_chains['SN_BAO_CMB'].get_chain()[Nmin:,:].reshape((-1, LCDM_chains['SN_Q_BAO_CMB'].get_chain().shape[-1]))
#corner(samples[:,:3], levels=(1-np.exp(-0.5*1),1-np.exp(-0.5*4)),
#             labels=labs, smooth=True, smooth1d=True, bins=25, plot_datapoints=False, color='k', 
#             fill_contours=True, contourf_kwargs=dict(colors=['white', 'lightgray', 'gray'], alpha=0.4),
#             fig=fig, range=[(0.15,.5), (0.03,0.07), (62,75)])

#plt.tight_layout()
#plt.tick_params(axis='both', which='major', labelsize=10)
#plt.tick_params(axis='both', which='minor', labelsize=8)


#plt.savefig('plots/posterior_oLCDM_full.pdf')


plt.show()

# Model 2: $w\Lambda$LCDM

## which data sets to include?

In [ ]:
model = 'wLCDM'
CMBdata = CMB_data(model, 'Planck18')

In [ ]:
data_sets = [SNdata, Qdata, BAOdata, CMBdata]
data_set_names = [set.name for set in data_sets]
data_sets_str = '_' + '_'.join([name for name in data_set_names]) + '_' 

## Define the likelihood and priors

In [ ]:
ranges_wLCDM_min=[0, 0, 0, 60., -2.5, -5, -10, -30, -.5, 0, 0]
ranges_wLCDM_max=[1, 1.5, 0.1, 80., -1/3., 5, 10, -10, .5, 10, 3]
def lnprob_wLCDM(theta):
    l = likelihood(theta, data_sets, ranges_wLCDM_min, ranges_wLCDM_max, model = 'wLCDM', rd_num = rd_num, z_num = z_num)
    return l.logprobability_gauss_prior()

## define a reference BIC

In [ ]:
BIC_wLCDM = lambda theta, data_sets_var: np.log(sum([len(d.get_data()) for d in data_sets_var])) * len(theta) -2*lnprob_wLCDM(theta)

## run the MCMC sampler  [arXiv:1202.3665]...

In [ ]:
ndim, nwalkers, nsteps = 10, 20, 100
pos0 = np.random.uniform(ranges_wLCDM_min, ranges_wLCDM_max, (nwalkers,len(ranges_wLCDM_max)))

# pool = Pool(cpu_count())
# write = HDFBackend('chains/LCDM_' + str(nwalkers) + 'x' + str(nsteps) + '.h5')
# sampler = EnsembleSampler(nwalkers, ndim, lnprob_wLCDM, pool=pool)#, backend=write)
# 
# sampler.run_mcmc();

## ...or load existing chains

In [ ]:
wLCDM_sampler = HDFBackend('chains/wLCDM' + data_sets_str + '512x1000.h5', read_only=True)
#LCDM_sampler = sampler

nsteps, nwalkers, ndim = wLCDM_sampler.get_chain().shape

labs = [r'$\Omega_m$', r'$\Omega_\Lambda$', r'$\Omega_b$', r'$H_0\ [\frac{\mathrm{km}/s}{\mathrm{Mpc}}]$', r'$w$', r'$\alpha$', r'$\beta$', r'$M_B$', r'$\Delta M_B$', r'$\beta^\prime$', r'$\delta$']

## Check and visualise the convergence: $\tau_f / n_\mathrm{steps} < 1/30$

In [ ]:
Nmin = 300
while Nmin < nsteps and np.all(wLCDM_sampler.get_autocorr_time(tol=0,discard=Nmin)/(nsteps)*30 > 1):
    Nmin+=100
    
print('Nmin = {}'.format(Nmin))

In [ ]:
wLCDM_samples = wLCDM_sampler.get_chain()

dellist = np.unique(np.where(np.isclose(wLCDM_samples[-1] - wLCDM_samples[0], 0))[0])
wLCDM_samples = np.delete(wLCDM_samples, np.unique(dellist), axis=1)

f, ax = plt.subplots(len(wLCDM_samples.T), 1, figsize=(6,3*len(wLCDM_samples.T)))

for i in range(len(wLCDM_samples.T)):
    ax[i].plot(wLCDM_samples.T[i].T, lw=0.5)
    ax[i].plot([Nmin, Nmin], [min(wLCDM_samples.T[i].flatten()), max(wLCDM_samples.T[i].flatten())], c='k', lw=1.5)
    ax[i].text(Nmin, min(wLCDM_samples.T[i].flatten()), r'$n_\mathrm{min}$', rotation=90, ha='right')
    ax[i].set_ylabel(labs[i])
    ax[i].set_xlabel('iteration')
    
    
plt.tight_layout()
#plt.savefig('plots/convergence_wLCDM.png')
plt.show()

## Plot the 1$\sigma$ and 2$\sigma$ contours

In [ ]:
wLCDM_samples = wLCDM_samples[Nmin:, :].reshape((-1, ndim))


meanwLCDM = np.mean(wLCDM_samples, axis=0)
stdwLCDM = np.var(wLCDM_samples, axis=0)
maxwLCDM=[]
for i in range(len(meanwLCDM)):
    like = np.histogram(wLCDM_samples.T[i], bins=50)
    i_max=np.argmax(like[0])
    max_val = (like[1][i_max]+like[1][i_max+1])/2
    maxwLCDM.append(max_val)

fig = corner(wLCDM_samples[:,:5], quantiles=(.16,.84),  levels=(1-np.exp(-0.5),1-np.exp(-0.5*4)),
             labels=labs, smooth=True, smooth1d=True, bins=30, plot_datapoints=False, fill_contours=True, contourf_kwargs=dict(colors=None, cmap='Greens'))#,
# #              truths=maxwLCDM)




plt.tight_layout()
#plt.savefig('plots/posterior_wLCDM' + data_sets_str[:-1] + '.pdf')

## Visualize the results

In [ ]:
v = np.percentile(wLCDM_samples,[16, 50, 84], axis = 0)
v = np.asarray([v[1], v[2]-v[1],v[1]-v[0]]).T

omegam, stdmp, stdmm = v[0]
omegac, stdcp, stdcm = v[1]
omegab, stdbp, stdbm = v[2]
H0, stdhp, stdhm = v[3]
w, wp, wm = v[4]


#model
z = SNdata.get_data().T[0]
# omegam, omegac, w = maxwLCDM[0], maxwLCDM[1], maxwLCDM[2]
best_fit_cosmo_wLCDM = cosmology(*maxwLCDM[:2], omegab = maxwLCDM[2], Hzero=maxwLCDM[3], w=maxwLCDM[4])
SNdata.set_param(v.T[0][-6:-2])
Qdata.set_param(v.T[0][-2:])

In [ ]:
bic_wlcdm = BIC_wLCDM(v.T[0],data_sets)

print('BIC(wLCDM) = ' + str(bic_wlcdm))

In [ ]:
fig = plt.figure(figsize=(8,8))
gs = gridspec.GridSpec(3, 1)
gs.update(hspace=0)


ax1 = plt.subplot(gs[0:2, 0])

if 'BAO'  in data_set_names or 'CMB' in data_set_names:    
    ax1.set_title(r'$\Omega_m = {%1.2f}^{+%1.2f}_{-%1.2f},\, \Omega_\Lambda = {%1.2f}^{+%1.2f}_{-%1.2f},\, w = {%1.2f}^{+%1.2f}_{-%1.2f},\, H_0 =  {%1.1f}^{+%1.1f}_{-%1.1f}\,\frac{\mathrm{km}/s}{\mathrm{Mpc}}$' % (omegam, stdmp,stdmm, omegac, stdcp,stdcm, w, wp, wm, H0, stdhp,stdhm), fontsize=17)
else:
    ax1.set_title(r'$\Omega_m = {%1.2f}^{+%1.2f}_{-%1.2f},\, \Omega_\Lambda = {%1.2f}^{+%1.2f}_{-%1.2f},\, w = ,%1.2f}^{+%1.2f}_{-%1.2f}$' % (omegam, stdmp,stdmm, omegac, stdcp,stdcm, w, wp, wm))

if 'Quasars' in data_set_names:
    ax1.errorbar(Qdata.distance_modulus().T[0], Qdata.distance_modulus().T[1], yerr=Qdata.delta_distance_modulus(), linestyle='none', marker='o', color='orange', ecolor='yellow', markersize=3, alpha=0.6, zorder=-1, label=r'quasars')
if 'SN' in data_set_names:
    ax1.errorbar(SNdata.distance_modulus().T[0], SNdata.distance_modulus().T[1], yerr=SNdata.delta_distance_modulus(), linestyle='none', marker='o', color='cyan', ecolor='blue', markersize=3, alpha=0.6, zorder=-1, label=r'SNe')
if 'BAO' in data_set_names:
    ax1.errorbar(BAOdata.distance_modulus(best_fit_cosmo_wLCDM).T[0], BAOdata.distance_modulus(best_fit_cosmo_wLCDM).T[1], yerr=BAOdata.delta_distance_modulus(best_fit_cosmo_wLCDM), linestyle='none', marker='o', color='red', ecolor='black', markersize=3, alpha=0.6, zorder=1, label=r'BAO')


zPlot = np.logspace(-3,np.log10(6),100)
ax1.plot(zPlot, best_fit_cosmo_wLCDM.distance_modulus(zPlot), c='k', label=r'best fit')
ax1.fill_between(zPlot, cosmology(omegam+stdmp,omegac+stdcp, omegab=omegab+stdbp, w = w+wp, Hzero=H0+stdhp).distance_modulus(zPlot), cosmology(omegam-stdmm,omegac-stdcm, omegab=omegab-stdbm, Hzero=H0-stdhm, w=w-wm).distance_modulus(zPlot), color='gray', alpha=0.3)
ax1.plot(zPlot, cosmology(Omega_m_preset, Omega_c_preset).distance_modulus(zPlot), ls = '--', c='gray', label=r'$\Lambda$CDM')

ax1.set_xlim(0.01,6)
ax1.set_ylim(32,55)


ax1.set_ylabel(r'distance modulus $\mu$')
ax1.set_xticklabels([])

ax1.grid('--', dashes=(5,5))
ax1.legend(loc=4)


ax2 = plt.subplot(gs[2, 0])

ax2.plot(zPlot, best_fit_cosmo_LCDM.distance_modulus(zPlot) - cosmology(Omega_m_preset, Omega_c_preset).distance_modulus(zPlot), 'k')

# ax2.set_xscale('log')
ax2.set_xlim(0.01,6)

ax2.text(0.97, .06, r'$\Delta\mathrm{BIC}(w\Lambda\mathrm{CDM}) = %1.0f$' % (BIC_wLCDM(v.T[0],data_sets) - bic_lcdm), ha='right', va='bottom', transform=ax2.transAxes, bbox=dict(facecolor=(1,1,1,.8), edgecolor='lightgray', boxstyle='round'))


ax2.grid('--', dashes=(5,5))
ax2.set_xlabel(r'$z$')
ax2.set_ylabel(r'$\mu_\mathrm{fit} - \mu_{\Lambda\mathrm{CDM}}$')

plt.tight_layout()
#plt.savefig('plots/Hubble_wLCDM' + data_sets_str[:-1] + '.pdf')
plt.show()

## Joint fit of all data sets

In [ ]:
wLCDM_chains = {}
wLCDM_chains['SN'] = HDFBackend('chains/wLCDM_SN_512x1000.h5')
wLCDM_chains['SN_Q'] = HDFBackend('chains/wLCDM_SN_Quasars_512x1000.h5')
wLCDM_chains['SN_Q_BAO'] = HDFBackend('chains/wLCDM_SN_Quasars_BAO_512x1000.h5')
wLCDM_chains['SN_Q_BAO_CMB'] = HDFBackend('chains/wLCDM_SN_Quasars_BAO_CMB_512x1000.h5')
wLCDM_chains['SN_BAO_CMB'] = HDFBackend('chains/wLCDM_SN_BAO_CMB_512x1000.h5')
wLCDM_chains['BAO'] = HDFBackend('chains/wLCDM_BAO_512x1000.h5')
wLCDM_chains['CMB'] = HDFBackend('chains/wLCDM_CMB_512x1000.h5')

In [ ]:
data_set_combinations = ['SN','SN_Q','SN_Q_BAO','SN_BAO_CMB','SN_Q_BAO_CMB','BAO','CMB']
data_set_combin_obs = {}
data_set_combin_obs['SN'] = [SNdata]
data_set_combin_obs['SN_Q'] = [SNdata,Qdata]
data_set_combin_obs['SN_Q_BAO'] = [SNdata,Qdata,BAOdata]
data_set_combin_obs['SN_BAO_CMB'] = [SNdata,BAOdata,CMBdata]
data_set_combin_obs['SN_Q_BAO_CMB'] = [SNdata,Qdata,BAOdata,CMBdata]
data_set_combin_obs['BAO'] = [BAOdata]
data_set_combin_obs['CMB'] = [CMBdata]

In [ ]:
for set_name in data_set_combinations:
    sample = wLCDM_chains[set_name].get_chain()
    Nmin = 300
    while Nmin < nsteps and np.all(wLCDM_chains[set_name].get_autocorr_time(tol=0,discard=Nmin)/(nsteps)*30 > 1):
        Nmin+=100
    dellist = np.unique(np.where(np.isclose(sample[-1] - sample[0], 0))[0])
    sample = np.delete(sample, np.unique(dellist), axis=1)[Nmin:, :].reshape((-1, ndim))

    data_sets = data_set_combin_obs[set_name]
    
    vtemp = np.percentile(sample,[16, 50, 84], axis = 0)
    vtemp = np.asarray([vtemp[1], vtemp[2]-vtemp[1],vtemp[1]-vtemp[0]]).T
    print(set_name + ':')
    print('   Om_m:' + str(np.round(vtemp[0],4)))
    print('   Om_c:' + str(np.round(vtemp[1],4)))
    print('   Om_b:' + str(np.round(vtemp[2],4)*100))
    print('   H0:  ' + str(np.round(vtemp[3],2)))
    print('   w:   ' + str(np.round(vtemp[4],3)))
    print('   dBIC:' + str(np.round(BIC_wLCDM(vtemp.T[0], data_set_combin_obs[set_name])-bic_lcdmlist[set_name])))

In [ ]:
Nmin = 300
for sampler in wLCDM_chains.values():
    nsteps, nwalkers, ndim = sampler.get_chain().shape
    while Nmin < nsteps and np.all(sampler.get_autocorr_time(tol=0,discard=Nmin)/(nsteps)*30 > 1):
        Nmin+=100
print('Nmin=',Nmin)

In [ ]:
samples = wLCDM_chains['SN'].get_chain()[Nmin:,np.delete(range(nwalkers), calcdellist(wLCDM_chains['SN'])),:].reshape((-1, wLCDM_chains['SN'].get_chain().shape[-1]))
fig=corner(samples[:,:5], levels=(1-np.exp(-0.5),1-np.exp(-0.5*4)),
            labels=labs, smooth=True, smooth1d=True, bins=25, plot_datapoints=False, color='green',
            fill_contours=True, contourf_kwargs=dict(colors=['white', 'lightgreen', 'green']),
            range=[(0,.6), (0,1.5), (0.03,0.07), (62,75), (-1.5, -0.5)])

samples = wLCDM_chains['SN_Q'].get_chain()[Nmin:,np.delete(range(nwalkers), calcdellist(wLCDM_chains['SN_Q'])),:].reshape((-1, wLCDM_chains['SN_Q'].get_chain().shape[-1]))
corner(samples[:,:5], levels=(1-np.exp(-0.5),1-np.exp(-0.5*4)),
            labels=labs, smooth=True, smooth1d=True, bins=25, plot_datapoints=False, color='blue', 
            fill_contours=True, contourf_kwargs=dict(colors=['white', 'lightblue', 'blue'], alpha=0.6),
            fig=fig, range=[(0,.6), (0,1.5), (0.03,0.07), (62,75), (-1.5, -0.5)])

samples = wLCDM_chains['CMB'].get_chain()[Nmin:,np.delete(range(nwalkers), calcdellist(wLCDM_chains['CMB'])),:].reshape((-1, wLCDM_chains['CMB'].get_chain().shape[-1]))
corner(samples[:,:5], levels=(1-np.exp(-0.5*1),1-np.exp(-0.5*4)),
            labels=labs, smooth=True, smooth1d=True, bins=25, plot_datapoints=False, color='orange', 
            fill_contours=True, contourf_kwargs=dict(colors=['white', 'yellow', 'orange'], alpha=0.4),
            fig=fig, range=[(0,.6), (0,1.5), (0.03,0.07), (62,75), (-1.5, -0.5)])

samples = wLCDM_chains['SN_Q_BAO_CMB'].get_chain()[Nmin:,np.delete(range(nwalkers), calcdellist(wLCDM_chains['SN_Q_BAO_CMB'])),:].reshape((-1, wLCDM_chains['SN_Q_BAO_CMB'].get_chain().shape[-1]))
corner(samples[:,:5], levels=(1-np.exp(-0.5*1),1-np.exp(-0.5*4)),
            labels=labs, smooth=True, smooth1d=True, bins=25, plot_datapoints=False, color='cyan', 
            fill_contours=True, contourf_kwargs=dict(colors=['white', 'lightblue', 'cyan'], alpha=0.4),
            fig=fig, range=[(0,.6), (0,1.5), (0.03,0.07), (62,75), (-1.5, -0.5)])

samples = wLCDM_chains['BAO'].get_chain()[Nmin:,np.delete(range(nwalkers), calcdellist(wLCDM_chains['BAO'])),:].reshape((-1, wLCDM_chains['BAO'].get_chain().shape[-1]))
fig=corner(samples[:,:5], levels=(1-np.exp(-0.5*1),1-np.exp(-0.5*4)),
            labels=labs, smooth=True, smooth1d=True, bins=25, plot_datapoints=False, color='red', 
            fill_contours=True, contourf_kwargs=dict(colors=['white', 'pink', 'red'], alpha=0.4),
            fig=fig, range=[(0,.6), (0,1.5), (0.03,0.07), (62,75), (-1.5, -0.5)])






# samples = wLCDM_chains['SN_BAO_CMB'].get_chain()[Nmin:,:].reshape((-1, wLCDM_chains['SN_BAO_CMB'].get_chain().shape[-1]))
# corner(samples[:,:5], levels=(1-np.exp(-0.5*1),1-np.exp(-0.5*4)),
#             labels=labs, smooth=True, smooth1d=True, bins=25, plot_datapoints=False, color='k', 
#             fill_contours=True, contourf_kwargs=dict(colors=['white', 'lightgray', 'gray'], alpha=0.4),
#             fig=fig, range=[(0,.6), (0,1.5), (0.03,0.07), (62,75), (-1.5, -0.5)])

plt.tight_layout()
plt.savefig('plots/posterior_wLCDM.pdf')


plt.show()

# Model 3: Bigravity

In [ ]:
model = 'bigravity'
CMBdata = CMB_data(model, 'Planck18')

In [ ]:
data_sets = [SNdata, Qdata, BAOdata, CMBdata]
data_set_names = [set.name for set in data_sets]
data_sets_str = '_' + '_'.join([name for name in data_set_names]) + '_' 

In [ ]:
# Testing the Bigravity scale factor function
#bigravity_cosmology(-33,1.5,1,2,3,Omega_m_preset,Omega_c_preset).Bianchi(.3)
#bigravity_cosmology(-33,1.5,1,2,-3,Omega_m_preset,Omega_c_preset).Bianchi(.3)
#bigravity_cosmology(-32,1,1,2,3,Omega_m_preset,Omega_c_preset).Bianchi(.1)
#bigravity_cosmology(-34,2,1,2,-3,Omega_m_preset,Omega_c_preset).Bianchi(1)

## Define the likelihood and priors

In [ ]:
ranges_bigravity_min=[-3,-3,-3, 0, .1, 0, 60, -5, -10, -30, -.5, 0, 0]
ranges_bigravity_max=[3, 3, 3, np.pi/2, .5, .1, 80, 5, 10, -10, .5, 10, 3]
def lnprob_bigravity(theta):
    l = likelihood(theta, data_sets, ranges_bigravity_min, ranges_bigravity_max, model = 'bigravity', rd_num = rd_num, z_num = z_num)
    return l.logprobability_gauss_prior()

## define a reference BIC

In [ ]:
BIC_bigravity = lambda theta, data_sets_var: np.log(sum([len(d.get_data()) for d in data_sets_var])) * len(theta) -2*lnprob_bigravity(theta)

## run the MCMC sampler  [arXiv:1202.3665]...

In [ ]:
ndim, nwalkers, nsteps = len(ranges_bigravity_max), 50, 100
pos0 = np.random.uniform(ranges_bigravity_min, ranges_bigravity_max, (nwalkers,len(ranges_bigravity_max)))


#pool = Pool(cpu_count())
#write = HDFBackend('chains/bigravity_' + str(nwalkers) + 'x' + str(nsteps) + '.h5')
#bigravity_sampler = EnsembleSampler(nwalkers, ndim, lnprob_bigravity, pool=pool)#, backend=write)

#bigravity_sampler.run_mcmc(pos0, nsteps, progress=True);

## ...or load existing chains

In [ ]:
bigravity_sampler = HDFBackend('chains/bigravity' + data_sets_str + '512x1000.h5', read_only=True)

nsteps, nwalkers, ndim = bigravity_sampler.get_chain().shape

labs = [r'$B_1$', r'$B_2$', r'$B_3$', r'$\theta$', r'$\Omega_m$',r'$\Omega_b$', r'$H_0\ [\frac{\mathrm{km}/s}{\mathrm{Mpc}}]$', r'$\alpha$', r'$\beta$', r'$M_B$', r'$\Delta M_B$', r'$\beta^\prime$', r'$\delta$']


## Check and visualise the convergence: $\tau_f / n_\mathrm{steps} < 1/30$

In [ ]:
Nmin = 800
while Nmin < nsteps and np.all(bigravity_sampler.get_autocorr_time(tol=0,discard=Nmin)/(nsteps)*30 > 1):
    Nmin+=100
    
print('Nmin = {}'.format(Nmin))

In [ ]:
bigravity_samples = bigravity_sampler.get_chain()

dellist = np.unique(np.where(np.isclose(bigravity_samples[-1] - bigravity_samples[0], 0))[0])

bigravity_samples = np.delete(bigravity_samples, np.unique(dellist), axis=1)

In [ ]:
f, ax = plt.subplots(len(bigravity_samples.T), 1, figsize=(6,3*len(bigravity_samples.T)))

for i in range(len(bigravity_samples.T)):
    ax[i].plot(bigravity_samples.T[i].T, lw=0.5)
#     ax[i].plot([Nmin, Nmin], [min(bigravity_samples.T[i].flatten()), max(bigravity_samples.T[i].flatten())], c='k', lw=1.5)
#     ax[i].text(Nmin, min(bigravity_samples.T[i].flatten()), r'$n_\mathrm{min}$', rotation=90, ha='right')
    ax[i].set_ylabel(labs[i])
    ax[i].set_xlabel('iteration')
    
    
plt.tight_layout()
#plt.savefig('plots/convergence_bigravity.png')
plt.show()

In [ ]:
Nmin = 800
while Nmin < nsteps and np.all(bigravity_sampler.get_autocorr_time(tol=0,discard=Nmin)/(nsteps)*30 > 1):
    Nmin+=100
    
print('Nmin = {}'.format(Nmin))

## Plot the 1$\sigma$ and 2$\sigma$ contours

In [ ]:
#if this fails w/ error "more dimensions than samples!", replace Nmin by a lower number for testing
#bigravity_samples = bigravity_samples[0:, :].reshape((-1, ndim))
bigravity_samples = bigravity_samples[Nmin:, :].reshape((-1, ndim))


meanbigravity = np.mean(bigravity_samples, axis=0)
stdbigravity = np.var(bigravity_samples, axis=0)
maxbigravity=[]
for i in range(len(meanbigravity)):
    like = np.histogram(bigravity_samples.T[i], bins=1000)
    i_max=np.argmax(like[0])
    max_val = (like[1][i_max]+like[1][i_max+1])/2
    maxbigravity.append(max_val)

fig = corner(bigravity_samples[:,:7], quantiles=(.16,.84),  levels=(1-np.exp(-0.5),1-np.exp(-0.5*4)),# range=[(0,0.8),(0.01,0.08), (60,80)],
             labels=labs, smooth=True, smooth1d=True, bins=20, plot_datapoints=False, fill_contours=True, contourf_kwargs=dict(colors=None, cmap='Greens'))#,
#              truths=maxLCDM)




plt.tight_layout()
#plt.savefig('plots/posterior_bigravity' + data_sets_str[:-1] + '.pdf')

## Visualize the results

In [ ]:
v = np.percentile(bigravity_samples,[16, 50, 84], axis = 0)
v = np.asarray([v[1], v[2]-v[1],v[1]-v[0]]).T

log10m, stdmgp, stdmgm = -32., 0, 0
beta1, stdb1p, stdb1m = v[0]
beta2, stdb2p, stdb2m = v[1]
beta3, stdb3p, stdb3m = v[2]
theta, stdtp, stdtm = v[3]
omegam, stdmp, stdmm = v[4]
omegab, stdbp, stdbm = v[5]
H0, stdhp, stdhm = v[6]

if ndim == 9:
    b1, b2, b3 = 1.,1.,-1.,1.
elif ndim >=9:
    b1, b2, b3 = v[0:3].T[0]


#model
z = SNdata.get_data().T[0]
best_fit_bigravitycosmo = bigravity_cosmology(log10m, theta, beta1, beta2, beta3, omegam,1-omegam-omega_gamma_preset, omegar=omega_gamma_preset, omegab=omegab, Hzero=H0)
SNdata.set_param(v.T[0][-6:-2])
Qdata.set_param(v.T[0][-2:])

# Calculate physical graviton mass of best fit cosmology:
mg = best_fit_bigravitycosmo.graviton_mass(1-omegam-omega_gamma_preset)
# Calculate error of physical graviton mass:
delmg = best_fit_bigravitycosmo.del_graviton_mass(1-omegam-omega_gamma_preset,stdhp,(stdb1p+stdb1m)/2,(stdb2p+stdb2m)/2,(stdb3p+stdb3m)/2)

In [ ]:
# Graviton mass in best fit bigravity:
print(mg,delmg)

In [ ]:
#Let's show that Bigravity reproduces LCDM at high-z using the best fit parameters:
test_cosmo_LCDM = cosmology(omegam, omegac=1-omegam, omegab=omegab, omegar=omega_gamma_preset, Hzero=H0)

# plot y(z) for best fit
z = np.arange(0.001, 1000.0, .01)

fig, ax = plt.subplots()
ax.semilogx(z, best_fit_bigravitycosmo.H(z)/test_cosmo_LCDM.H(z))

ax.set_title('$H(z)$ of bigravity / $H(z)$ of $\Lambda$CDM ', fontsize=18)

ax.set(xlabel='z')
ax.set_xlim(0.001,1000)
plt.xticks((.001,.01,.1,1,10,100,1000))
#plt.yticks(np.arange(1, 1.025, .01))
#ax.get_xaxis().set_major_formatter(FormatStrFormatter())
ax.yaxis.set_minor_formatter(FormatStrFormatter('%.0f'))
ax.grid('--',which='major',axis='x', dashes=(5,5))
ax.grid('--',which='both',axis='y', dashes=(5,5))


#ax.grid('--', dashes=(5,5))
plt.tight_layout()
plt.show()

plt.savefig('plots/bigra_best_fit_H_over_LCDM.pdf')

In [ ]:
#Show that y at z=0 is approx equal to ystar = y(z=-1)
(best_fit_bigravitycosmo.Bianchi(0),best_fit_bigravitycosmo.Bianchi(-1))

In [ ]:
# plot y(z) for best fit
z = np.arange(0.1, 1000.0, 0.01)

fig, ax = plt.subplots()
ax.semilogx(z, best_fit_bigravitycosmo.Bianchi(z))

ax.set_title('Bigravity best fit cosmology', fontsize=18)

ax.set(xlabel='z', ylabel='y(z)')
ax.grid()
plt.tight_layout()

plt.show()
#plt.savefig('plots/bigra_best_fit_y' + data_sets_str[:-1] + '.pdf')

In [ ]:
bic_lcdm = 1442

In [ ]:
bic_bi - bic_lcdm

In [ ]:
fig = plt.figure(figsize=(8,8))
gs = gridspec.GridSpec(3, 1)
gs.update(hspace=0)


ax1 = plt.subplot(gs[0:2, 0])

if 'BAO'  in data_set_names or 'CMB' in data_set_names:    
    ax1.set_title(r'$\Omega_m = {%1.2f}^{+%1.2f}_{-%1.2f},\, H_0 =  {%1.1f}^{+%1.1f}_{-%1.1f}\,\frac{\mathrm{km}/s}{\mathrm{Mpc}}$' % (omegam, stdmp,stdmm, H0, stdhp,stdhm), fontsize=18)
else:
    ax1.set_title(r'$\Omega_m = {%1.2f}^{+%1.2f}_{-%1.2f}$' % (omegam, stdmp,stdmm), fontsize=18)


ax1.text(0.5, 0.96, r'$\beta_{1,2,3} = \left({%1.1f}^{+%1.1f}_{-%1.1f}, {%1.1f}^{+%1.1f}_{-%1.1f},{%1.1f}^{+%1.1f}_{-%1.1f}\right),\,m_g = ({%1.2f}\pm {%1.2f})\times 10^{%1.0f} \,\mathrm{eV}$' % (b1, stdb1p, stdb1m, b2, stdb2p, stdb2m, b3, stdb3p, stdb3m, mg/(10**np.floor(np.log10(mg))),delmg/(10**np.floor(np.log10(mg))), np.floor(np.log10(mg))),
         ha='center', va='top', fontsize=16, transform=ax1.transAxes, bbox=dict(facecolor=(1,1,1,.8), edgecolor='lightgray', boxstyle='round'))
   
if 'Quasars' in data_set_names:
    ax1.errorbar(Qdata.distance_modulus().T[0], Qdata.distance_modulus().T[1], yerr=Qdata.delta_distance_modulus(), linestyle='none', marker='o', color='orange', ecolor='yellow', markersize=3, alpha=0.6, zorder=-1, label=r'quasars')
if 'SN' in data_set_names:
    ax1.errorbar(SNdata.distance_modulus().T[0], SNdata.distance_modulus().T[1], yerr=SNdata.delta_distance_modulus(), linestyle='none', marker='o', color='cyan', ecolor='blue', markersize=3, alpha=0.6, zorder=-1, label=r'SNe')
if 'BAO' in data_set_names:
    ax1.errorbar(BAOdata.distance_modulus(best_fit_bigravitycosmo).T[0], BAOdata.distance_modulus(best_fit_bigravitycosmo).T[1], yerr=BAOdata.delta_distance_modulus(best_fit_bigravitycosmo), linestyle='none', marker='o', color='red', ecolor='black', markersize=3, alpha=0.6, zorder=-1, label=r'BAO')



zPlot = np.logspace(-3,np.log10(6),100)
ax1.plot(zPlot, best_fit_bigravitycosmo.distance_modulus(zPlot), c='k', label=r'fit')
ax1.fill_between(zPlot, bigravity_cosmology(log10m+stdmp, theta + stdtp, beta1+stdb1p,beta2+stdb2p,beta3+stdb3p, omegam + stdmp, 1- omegam - stdmp, omegab=omegab+stdbp, Hzero=H0+stdhp).distance_modulus(zPlot), bigravity_cosmology(log10m-stdmm, theta - stdtm, beta1 - stdb1m,beta2 - stdb2m, beta3 - stdb3m, omegam - stdmm, 1-omegam + stdmm, omegab=omegab-stdbm, Hzero=H0-stdhm).distance_modulus(zPlot), color='gray', alpha=0.3)
ax1.plot(zPlot, cosmology(Omega_m_preset, Omega_c_preset, Hzero=H0).distance_modulus(zPlot), ls = '--', c='gray', label=r'$\Lambda$CDM')

ax1.set_xlim(0.01,6)
ax1.set_ylim(32,55)


ax1.set_ylabel(r'distance modulus $\mu$')
ax1.set_xticklabels([])

ax1.grid('--', dashes=(5,5))
ax1.legend(loc=4, fontsize=16)


ax2 = plt.subplot(gs[2, 0])

ax2.plot(zPlot, best_fit_bigravitycosmo.distance_modulus(zPlot) - cosmology(Omega_m_preset, Omega_c_preset).distance_modulus(zPlot), 'k')

# ax2.set_xscale('log')
ax2.set_xlim(0.01,6)

ax2.text(0.97, .93, r'$\Delta \mathrm{BIC}(\mathrm{bigravity}) = %1.0f$' % (BIC_bigravity(v.T[0],data_sets) - bic_lcdm), ha='right', va='top', transform=ax2.transAxes, bbox=dict(facecolor=(1,1,1,.8), edgecolor='lightgray', boxstyle='round'))


ax2.grid('--', dashes=(5,5))
ax2.set_xlabel(r'$z$')
ax2.set_ylabel(r'$\mu_\mathrm{fit} - \mu_{\Lambda\mathrm{CDM}}$')

plt.tight_layout()
plt.savefig('plots/Hubble_bigravity' + data_sets_str[:-1] + '.pdf')
plt.show()

## Joint fit of all data sets

In [ ]:
#Load all chains
bigravity_chains = {}
bigravity_chains['SN'] = HDFBackend('chains/bigravity_SN_512x1000.h5')
bigravity_chains['SN_Q'] = HDFBackend('chains/bigravity_SN_Quasars_512x1000.h5')
bigravity_chains['SN_Q_BAO'] = HDFBackend('chains/bigravity_SN_Quasars_BAO_512x1000.h5')
bigravity_chains['SN_Q_BAO_CMB'] = HDFBackend('chains/bigravity_SN_Quasars_BAO_CMB_512x1000.h5')
bigravity_chains['CMB'] = HDFBackend('chains/bigravity_CMB_512x1000.h5')
bigravity_chains['BAO'] = HDFBackend('chains/bigravity_BAO_512x1000.h5')

In [ ]:
data_set_combinations = ['SN','SN_Q','SN_Q_BAO','SN_Q_BAO_CMB','BAO','CMB']
data_set_combin_obs = {}
data_set_combin_obs['SN'] = [SNdata]
data_set_combin_obs['SN_Q'] = [SNdata,Qdata]
data_set_combin_obs['SN_Q_BAO'] = [SNdata,Qdata,BAOdata]
#data_set_combin_obs['SN_BAO_CMB'] = [SNdata,BAOdata,CMBdata]
data_set_combin_obs['SN_Q_BAO_CMB'] = [SNdata,Qdata,BAOdata,CMBdata]
data_set_combin_obs['BAO'] = [BAOdata]
data_set_combin_obs['CMB'] = [CMBdata]

In [ ]:
for set_name in data_set_combinations:
    sample = bigravity_chains[set_name].get_chain()
    Nmin = 800
    while Nmin < nsteps and np.all(bigravity_chains[set_name].get_autocorr_time(tol=0,discard=Nmin)/(nsteps)*30 > 1):
        Nmin+=100
    dellist = np.unique(np.where(np.isclose(sample[-1] - sample[0], 0))[0])
    sample = np.delete(sample, np.unique(dellist), axis=1)[Nmin:, :].reshape((-1, ndim))
    
    data_sets = data_set_combin_obs[set_name]
    
    vtemp = np.percentile(sample,[16, 50, 84], axis = 0)
    vtemp = np.asarray([vtemp[1], vtemp[2]-vtemp[1],vtemp[1]-vtemp[0]]).T
    print(set_name + ':')
    print('   Om_m:   ' + str(np.round(vtemp[4],4)))
    print('   Om_b:   ' + str(np.round(vtemp[5],4)*100))
    print('   H0:     ' + str(np.round(vtemp[6],2)))
    print('   Theta:  ' + str(np.round(vtemp[3],3)))
    print('   b1:     ' + str(np.round(vtemp[0],3)))
    print('   b2:     ' + str(np.round(vtemp[1],3)))
    print('   b3:     ' + str(np.round(vtemp[2],3)))
    print('   dBIC:   ' + str(np.round(BIC_bigravity(vtemp.T[0], data_set_combin_obs[set_name])-bic_lcdmlist[set_name])))
    #print(vtemp.T[0])

In [ ]:
Nmin = 600
for sampler in bigravity_chains.values():
    nsteps, nwalkers, ndim = sampler.get_chain().shape
    while Nmin < nsteps and np.all(sampler.get_autocorr_time(tol=0,discard=Nmin)/(nsteps)*30 > 1):
        Nmin+=100
print("Nmin = ", Nmin)

In [ ]:
samples = bigravity_chains['SN'].get_chain()[Nmin:,np.delete(range(nwalkers), calcdellist(bigravity_chains['SN'])),:].reshape((-1, bigravity_chains['SN'].get_chain().shape[-1]))
fig=corner(samples[:,4:7], levels=(1-np.exp(-0.5),1-np.exp(-0.5*4)),
            labels=labs[4:7], smooth=True, smooth1d=True, bins=25, plot_datapoints=False, color='green',
            fill_contours=True, contourf_kwargs=dict(colors=['white', 'lightgreen', 'green']),
            range=[(0,1), (0.03,0.07), (60,73)])

samples = bigravity_chains['SN_Q'].get_chain()[Nmin:,np.delete(range(nwalkers), calcdellist(bigravity_chains['SN_Q'])),:].reshape((-1, bigravity_chains['SN_Q'].get_chain().shape[-1]))
corner(samples[:,4:7], levels=(1-np.exp(-0.5),1-np.exp(-0.5*4)),
            labels=labs[4:7], smooth=True, smooth1d=True, bins=25, plot_datapoints=False, color='blue', 
            fill_contours=True, contourf_kwargs=dict(colors=['white', 'lightblue', 'blue'], alpha=0.6),
            fig=fig, range=[(0,1), (0.03,0.07), (60,73)])

samples = bigravity_chains['CMB'].get_chain()[Nmin:,np.delete(range(nwalkers), calcdellist(bigravity_chains['CMB'])),:].reshape((-1, bigravity_chains['CMB'].get_chain().shape[-1]))
corner(samples[:,4:7], levels=(1-np.exp(-0.5*1),1-np.exp(-0.5*4)),
           labels=labs[4:7], smooth=True, smooth1d=True, bins=25, plot_datapoints=False, color='orange', 
           fill_contours=True, contourf_kwargs=dict(colors=['white', 'yellow', 'orange'], alpha=0.4),
           fig=fig, range=[(0,1), (0.03,0.07), (60,73)])

samples = bigravity_chains['BAO'].get_chain()[Nmin:,np.delete(range(nwalkers), calcdellist(bigravity_chains['BAO'])),:].reshape((-1, bigravity_chains['BAO'].get_chain().shape[-1]))
corner(samples[:,4:7], levels=(1-np.exp(-0.5*1),1-np.exp(-0.5*4)),
            labels=labs[4:7], smooth=True, smooth1d=True, bins=25, plot_datapoints=False, color='red', 
            fill_contours=True, contourf_kwargs=dict(colors=['white', 'pink', 'red'], alpha=0.4),
            fig=fig, range=[(0,1), (0.03,0.07), (60,73)])

samples = bigravity_chains['SN_Q_BAO_CMB'].get_chain()[Nmin:,np.delete(range(nwalkers), calcdellist(bigravity_chains['SN_Q_BAO_CMB'])),:].reshape((-1, bigravity_chains['SN_Q_BAO_CMB'].get_chain().shape[-1]))
corner(samples[:,4:7], levels=(1-np.exp(-0.5),1-np.exp(-0.5*4)),
            labels=labs[4:7], smooth=True, smooth1d=True, bins=25, plot_datapoints=False, color='cyan',
            fill_contours=True, contourf_kwargs=dict(colors=['white', 'lightgreen', 'green'], alpha=0.4),
            fig=fig, range=[(0,1), (0.03,0.07), (60,73)])


plt.tight_layout()
plt.savefig('plots/posterior_bigravity.pdf')


plt.show()

# Model 4: oBigravity

In [ ]:
model = 'obigravity'
CMBdata = CMB_data(model, 'Planck18')

In [ ]:
data_sets = [SNdata, Qdata, BAOdata, CMBdata]
data_set_names = [set.name for set in data_sets]
data_sets_str = '_' + '_'.join([name for name in data_set_names]) + '_' 

## Define the likelihood and priors

In [ ]:
ranges_obigravity_min=[-3,-3,-3, 0, 0, 0, 0, 60, -5, -10, -30, -.5, 0, 0]
ranges_obigravity_max=[3, 3, 3, np.pi/2, 1, 1.5, .1, 80, 5, 10, -10, .5, 10, 3]
def lnprob_obigravity(theta):
    l = likelihood(theta, data_sets, ranges_obigravity_min, ranges_obigravity_max, model = 'obigravity', rd_num = rd_num, z_num = z_num)
    return l.logprobability_gauss_prior()

## define a reference BIC

In [ ]:
BIC_obigravity = lambda theta, data_sets_var: np.log(sum([len(d.get_data()) for d in data_sets_var])) * len(theta) -2*lnprob_obigravity(theta)

## run the MCMC sampler  [arXiv:1202.3665]...

In [ ]:
ndim, nwalkers, nsteps = len(ranges_obigravity_max), 50, 100
pos0 = np.random.uniform(ranges_obigravity_min, ranges_obigravity_max, (nwalkers,len(ranges_obigravity_max)))


#pool = Pool(cpu_count())
#write = HDFBackend('chains/obigravity_' + str(nwalkers) + 'x' + str(nsteps) + '.h5')
#obigravity_sampler = EnsembleSampler(nwalkers, ndim, lnprob_obigravity, pool=pool)#, backend=write)

#obigravity_sampler.run_mcmc(pos0, nsteps, progress=True);

## ...or load existing chains

In [ ]:
obigravity_sampler = HDFBackend('chains/obigravity' + data_sets_str + '512x1000.h5', read_only=True)

nsteps, nwalkers, ndim = obigravity_sampler.get_chain().shape

labs = [r'$B_1$', r'$B_2$', r'$B_3$', r'$\theta$', r'$\Omega_m$',  r'$\Omega_\Lambda$', r'$\Omega_b$', r'$H_0\ [\frac{\mathrm{km}/s}{\mathrm{Mpc}}]$', r'$\alpha$', r'$\beta$', r'$M_B$', r'$\Delta M_B$', r'$\beta^\prime$', r'$\delta$']

## Check and visualise the convergence: $\tau_f / n_\mathrm{steps} < 1/30$

In [ ]:
Nmin = 800
while Nmin < nsteps and np.all(obigravity_sampler.get_autocorr_time(tol=0,discard=Nmin)/(nsteps)*30 > 1):
    Nmin+=100
    
print('Nmin = {}'.format(Nmin))

In [ ]:
obigravity_samples = obigravity_sampler.get_chain()

dellist = np.unique(np.where(np.isclose(obigravity_samples[-1] - obigravity_samples[0], 0))[0])

obigravity_samples = np.delete(obigravity_samples, np.unique(dellist), axis=1)


f, ax = plt.subplots(len(obigravity_samples.T), 1, figsize=(6,3*len(obigravity_samples.T)))

for i in range(len(obigravity_samples.T)):
    ax[i].plot(obigravity_samples.T[i].T, lw=0.5)
#     ax[i].plot([Nmin, Nmin], [min(obigravity_samples.T[i].flatten()), max(obigravity_samples.T[i].flatten())], c='k', lw=1.5)
#     ax[i].text(Nmin, min(obigravity_samples.T[i].flatten()), r'$n_\mathrm{min}$', rotation=90, ha='right')
    ax[i].set_ylabel(labs[i])
    ax[i].set_xlabel('iteration')
    
    
plt.tight_layout()
plt.savefig('plots/convergence_obigravity.png')
plt.show()

In [ ]:
dellist = np.unique(np.where(np.isclose(obigravity_samples[-1] - obigravity_samples[0], 0))[0])

obigravity_samples = np.delete(obigravity_samples, np.unique(dellist), axis=1)

## Plot the 1$\sigma$ and 2$\sigma$ contours

In [ ]:
obigravity_samples = obigravity_samples[Nmin:, :].reshape((-1, ndim))


meanobigravity = np.mean(obigravity_samples, axis=0)
stdobigravity = np.var(obigravity_samples, axis=0)
maxobigravity=[]
for i in range(len(meanobigravity)):
    like = np.histogram(obigravity_samples.T[i], bins=1000)
    i_max=np.argmax(like[0])
    max_val = (like[1][i_max]+like[1][i_max+1])/2
    maxobigravity.append(max_val)

fig = corner(obigravity_samples[:,:8], quantiles=(.16,.84),  levels=(1-np.exp(-0.5),1-np.exp(-0.5*4)),
             labels=labs, smooth=True, smooth1d=True, bins=25, plot_datapoints=False, fill_contours=True, contourf_kwargs=dict(colors=None, cmap='Greens'))#,
# #              truths=maxobigravity)




plt.tight_layout()
plt.savefig('plots/posterior_obigravity' + data_sets_str[:-1] + '.pdf')

## Visualize the results

In [ ]:
v = np.percentile(obigravity_samples,[16, 50, 84], axis = 0)
v = np.asarray([v[1], v[2]-v[1],v[1]-v[0]]).T

log10m, stdmgp, stdmgm = -32., 0, 0
b1, stdb1p, stdb1m = v[0]
b2, stdb2p, stdb2m = v[1]
b3, stdb3p, stdb3m = v[2]
theta, stdtp, stdtm = v[3]
omegam, stdmp, stdmm = v[4]
omegac, stdcp, stdcm = v[5]
omegab, stdbp, stdbm = v[6]
H0, stdhp, stdhm = v[7]

if ndim == 9:
    b1, b2, b3 = 1.,1.,-1.,1.
elif ndim >=9:
    b1, b2, b3 = v[0:3].T[0]


#model
z = SNdata.get_data().T[0]
best_fit_obigravitycosmo = bigravity_cosmology(log10m, theta, b1, b2, b3, omegam, omegac, omegar=omega_gamma_preset, omegab=omegab, Hzero=H0)
SNdata.set_param(v.T[0][-6:-2])
Qdata.set_param(v.T[0][-2:])

# Calculate physical graviton mass of best fit cosmology:
mg = best_fit_obigravitycosmo.graviton_mass(omegac)
# Calculate error of physical graviton mass:
delmg = best_fit_obigravitycosmo.del_graviton_mass(omegac,stdhp,(stdb1p+stdb1m)/2,(stdb2p+stdb2m)/2,(stdb3p+stdb3m)/2)

In [ ]:
print(mg,delmg)

In [ ]:
#Let's show that oBigravity reproduces oLCDM at high-z using the best fit parameters:
test_cosmo_oLCDM = cosmology(omegam, omegac, omegab=omegab, omegar=omega_gamma_preset, Hzero=H0)

# plot y(z) for best fit
z = np.arange(0.001, 1000.0, .01)

fig, ax = plt.subplots()
ax.semilogx(z, best_fit_obigravitycosmo.H(z)/test_cosmo_oLCDM.H(z))

ax.set_title('$H(z)$ of oBigravity / $H(z)$ of o$\Lambda$CDM', fontsize=18)

ax.set(xlabel='z')
ax.set_xlim(0.001,1000)
ax.set_ylim(.984, 1.001,)
plt.xticks((.001,.01,.1,1,10,100,1000))
plt.yticks(np.arange(.985, 1.001, .005))
ax.yaxis.set_major_formatter(FormatStrFormatter('%.3f'))
ax.yaxis.set_minor_formatter(FormatStrFormatter('%.3f'))
ax.grid('--',which='major',axis='x', dashes=(5,5))
ax.grid('--',which='both',axis='y', dashes=(5,5))

#ax.grid('--', dashes=(5,5))
plt.tight_layout()

plt.show()
plt.savefig('plots/obigra_best_fit_H_over_oLCDM.pdf')

In [ ]:
(best_fit_obigravitycosmo.Bianchi(0),best_fit_obigravitycosmo.Bianchi(-1))

In [ ]:
# plot y(z) for best fit parameters
z = np.arange(-1, 30, 0.01)

fig, ax = plt.subplots()
ax.plot(z, best_fit_obigravitycosmo.Bianchi(z))

ax.set_title('oBigravity best fit cosmology', fontsize=18)

ax.set(xlabel='z', ylabel='y(z)')
ax.grid()
plt.tight_layout()

plt.show()
#plt.savefig('plots/obigra_best_fit_y' + data_sets_str[:-1] + '.pdf')

In [ ]:
bic_obi = BIC_obigravity(v.T[0],data_sets)

print('BIC(oBigravity) = ' + str(bic_obi))

In [ ]:
bic_obi -bic_lcdm

In [ ]:
fig = plt.figure(figsize=(8,8))
gs = gridspec.GridSpec(3, 1)
gs.update(hspace=0)


ax1 = plt.subplot(gs[0:2, 0])

if 'BAO'  in data_set_names or 'CMB' in data_set_names:    
    ax1.set_title(r'$\Omega_m = {%1.2f}^{+%1.2f}_{-%1.2f},\, \Omega_\Lambda = {%1.2f}^{+%1.2f}_{-%1.2f},\, H_0 =  {%1.1f}^{+%1.1f}_{-%1.1f}\,\frac{\mathrm{km}/s}{\mathrm{Mpc}}$' % (omegam, stdmp,stdmm, omegac, stdcp,stdcm, H0, stdhp,stdhm), fontsize=18)
else:
    ax1.set_title(r'$\Omega_m = {%1.2f}^{+%1.2f}_{-%1.2f}\,,\Omega_\Lambda = {%1.2f}^{+%1.2f}_{-%1.2f}$' % (omegam, stdmp,stdmm, omegac, stdcp,stdcm), fontsize=18)


ax1.text(0.5, 0.96, r'$\beta_{1,2,3} = \left({%1.1f}^{+%1.1f}_{-%1.1f}, {%1.1f}^{+%1.1f}_{-%1.1f},{%1.1f}^{+%1.1f}_{-%1.1f}\right),\, m_g = ({%1.2f} \pm {%1.2f})\times 10^{%1.0f} \,\mathrm{eV}$' % (b1, stdb1p, stdb1m, b2, stdb2p, stdb2m, b3, stdb3p, stdb3m,mg/(10**np.floor(np.log10(mg))),delmg/(10**np.floor(np.log10(mg))), np.floor(np.log10(mg))), ha='center', va='top', fontsize=16, transform=ax1.transAxes, bbox=dict(facecolor=(1,1,1,.8), edgecolor='lightgray', boxstyle='round'))
   
if 'Quasars' in data_set_names:
    ax1.errorbar(Qdata.distance_modulus().T[0], Qdata.distance_modulus().T[1], yerr=Qdata.delta_distance_modulus(), linestyle='none', marker='o', color='orange', ecolor='yellow', markersize=3, alpha=0.6, zorder=-1, label=r'quasars')
if 'SN' in data_set_names:
    ax1.errorbar(SNdata.distance_modulus().T[0], SNdata.distance_modulus().T[1], yerr=SNdata.delta_distance_modulus(), linestyle='none', marker='o', color='cyan', ecolor='blue', markersize=3, alpha=0.6, zorder=-1, label=r'SNe')
if 'BAO' in data_set_names:
    ax1.errorbar(BAOdata.distance_modulus(best_fit_obigravitycosmo).T[0], BAOdata.distance_modulus(best_fit_obigravitycosmo).T[1], yerr=BAOdata.delta_distance_modulus(best_fit_obigravitycosmo), linestyle='none', marker='o', color='red', ecolor='black', markersize=3, alpha=0.6, zorder=-1, label=r'BAO')



zPlot = np.logspace(-3,np.log10(6),100)
ax1.plot(zPlot, best_fit_obigravitycosmo.distance_modulus(zPlot), c='k', label=r'fit')
ax1.fill_between(zPlot, bigravity_cosmology(log10m+stdmp, theta + stdtp, b1+stdb1p,b2+stdb2p,b3+stdb3p, omegam + stdmp, omegac + stdcp, omegab=omegab+stdbp, Hzero=H0+stdhp).distance_modulus(zPlot), bigravity_cosmology(log10m-stdmm, theta - stdtm, b1 - stdb1m,b2 - stdb2m, b3 - stdb3m, omegam - stdmm, omegac - stdcm, omegab=omegab-stdbm, Hzero=H0-stdhm).distance_modulus(zPlot), color='gray', alpha=0.3)
ax1.plot(zPlot, cosmology(Omega_m_preset, Omega_c_preset, Hzero=H0).distance_modulus(zPlot), ls = '--', c='gray', label=r'$\Lambda$CDM')
ax1.plot(zPlot, cosmology(omegam, omegac, Hzero=H0).distance_modulus(zPlot), ls = ':', c='lightgray', label=r'$\Lambda\mathrm{CDM}(\Omega_m, \Omega_\Lambda)$')

ax1.set_xlim(0.01,6)
ax1.set_ylim(32,55)


ax1.set_ylabel(r'distance modulus $\mu$')
ax1.set_xticklabels([])

ax1.grid('--', dashes=(5,5))
ax1.legend(loc=4, fontsize=16)


ax2 = plt.subplot(gs[2, 0])

ax2.plot(zPlot, best_fit_obigravitycosmo.distance_modulus(zPlot) - cosmology(Omega_m_preset, Omega_c_preset).distance_modulus(zPlot), 'k')

# ax2.set_xscale('log')
ax2.set_xlim(0.01,6)

ax2.text(0.97, .06, r'$\Delta \mathrm{BIC}(\mathrm{oBigravity}) = %1.0f$' % (BIC_obigravity(v.T[0],data_sets) - bic_lcdm), ha='right', va='bottom', transform=ax2.transAxes, bbox=dict(facecolor=(1,1,1,.8), edgecolor='lightgray', boxstyle='round'))


ax2.grid('--', dashes=(5,5))
ax2.set_xlabel(r'$z$')
ax2.set_ylabel(r'$\mu_\mathrm{fit} - \mu_{\Lambda\mathrm{CDM}}$')

plt.tight_layout()
plt.savefig('plots/Hubble_obigravity' + data_sets_str[:-1] + '.pdf')
plt.show()

## Joint fit of all data sets

In [ ]:
obigravity_chains = {}
obigravity_chains['SN'] = HDFBackend('chains/obigravity_SN_512x1000.h5')
obigravity_chains['SN_Q'] = HDFBackend('chains/obigravity_SN_Quasars_512x1000.h5')
obigravity_chains['SN_Q_BAO'] = HDFBackend('chains/obigravity_SN_Quasars_BAO_512x1000.h5')
obigravity_chains['SN_Q_BAO_CMB'] = HDFBackend('chains/obigravity_SN_Quasars_BAO_CMB_512x1000.h5')
obigravity_chains['CMB'] = HDFBackend('chains/obigravity_CMB_512x1000.h5')
obigravity_chains['BAO'] = HDFBackend('chains/obigravity_BAO_512x1000.h5')

In [ ]:
data_set_combinations = ['SN','SN_Q','SN_Q_BAO','SN_Q_BAO_CMB','BAO','CMB']
data_set_combin_obs = {}
data_set_combin_obs['SN'] = [SNdata]
data_set_combin_obs['SN_Q'] = [SNdata,Qdata]
data_set_combin_obs['SN_Q_BAO'] = [SNdata,Qdata,BAOdata]
#data_set_combin_obs['SN_BAO_CMB'] = [SNdata,BAOdata,CMBdata]
data_set_combin_obs['SN_Q_BAO_CMB'] = [SNdata,Qdata,BAOdata,CMBdata]
data_set_combin_obs['BAO'] = [BAOdata]
data_set_combin_obs['CMB'] = [CMBdata]

In [ ]:
for set_name in data_set_combinations:
    sample = obigravity_chains[set_name].get_chain()
    Nmin = 800
    while Nmin < nsteps and np.all(obigravity_chains[set_name].get_autocorr_time(tol=0,discard=Nmin)/(nsteps)*30 > 1):
        Nmin+=100
    dellist = np.unique(np.where(np.isclose(sample[-1] - sample[0], 0))[0])
    sample = np.delete(sample, np.unique(dellist), axis=1)[Nmin:, :].reshape((-1, ndim))
    
    data_sets = data_set_combin_obs[set_name]
    
    vtemp = np.percentile(sample,[16, 50, 84], axis = 0)
    vtemp = np.asarray([vtemp[1], vtemp[2]-vtemp[1],vtemp[1]-vtemp[0]]).T
    print(set_name + ':')
    print('   Om_m:   ' + str(np.round(vtemp[4],4)))
    print('   Om_c:   ' + str(np.round(vtemp[5],4)))
    print('   Om_b:   ' + str(np.round(vtemp[6],4)*100))
    print('   H0:     ' + str(np.round(vtemp[7],2)))
    print('   Theta:  ' + str(np.round(vtemp[3],3)))
    print('   b1:     ' + str(np.round(vtemp[0],3)))
    print('   b2:     ' + str(np.round(vtemp[1],3)))
    print('   b3:     ' + str(np.round(vtemp[2],3)))
    print('   dBIC:   ' + str(np.round(BIC_obigravity(vtemp.T[0], data_set_combin_obs[set_name])-bic_lcdmlist[set_name])))
    #print(Nmin)

In [ ]:
Nmin = 800
for sampler in obigravity_chains.values():
    nsteps, nwalkers, ndim = sampler.get_chain().shape
    while Nmin < nsteps and np.all(sampler.get_autocorr_time(tol=0,discard=Nmin)/(nsteps)*30 > 1):
        Nmin+=100
print("Nmin = ", Nmin)

In [ ]:
samples = obigravity_chains['SN'].get_chain()[Nmin:,np.delete(range(nwalkers), calcdellist(obigravity_chains['SN'])),:].reshape((-1, obigravity_chains['SN'].get_chain().shape[-1]))
fig=corner(samples[:,4:8], levels=(1-np.exp(-0.5),1-np.exp(-0.5*4)),
            labels=labs[4:8], smooth=True, smooth1d=True, bins=25, plot_datapoints=False, color='green',
            fill_contours=True, contourf_kwargs=dict(colors=['white', 'lightgreen', 'green']),
            range=[(0,1), (0,1.25), (0.03,0.07), (60,75)])

samples = obigravity_chains['SN_Q'].get_chain()[Nmin:,np.delete(range(nwalkers), calcdellist(obigravity_chains['SN_Q'])),:].reshape((-1, obigravity_chains['SN_Q'].get_chain().shape[-1]))
corner(samples[:,4:8], levels=(1-np.exp(-0.5),1-np.exp(-0.5*4)),
            labels=labs[4:8], smooth=True, smooth1d=True, bins=25, plot_datapoints=False, color='blue', 
            fill_contours=True, contourf_kwargs=dict(colors=['white', 'lightblue', 'blue'], alpha=0.6),
            fig=fig, range=[(0,1), (0,1.25), (0.03,0.07), (60,75)])

samples = obigravity_chains['CMB'].get_chain()[Nmin:,np.delete(range(nwalkers), calcdellist(obigravity_chains['CMB'])),:].reshape((-1, obigravity_chains['CMB'].get_chain().shape[-1]))
corner(samples[:,4:8], levels=(1-np.exp(-0.5*1),1-np.exp(-0.5*4)),
            labels=labs[4:8], smooth=True, smooth1d=True, bins=25, plot_datapoints=False, color='orange', 
            fill_contours=True, contourf_kwargs=dict(colors=['white', 'yellow', 'orange'], alpha=0.4),
            fig=fig, range=[(0,1), (0,1.25), (0.03,0.07), (60,75)])

samples = obigravity_chains['SN_Q_BAO_CMB'].get_chain()[Nmin:,np.delete(range(nwalkers), calcdellist(obigravity_chains['SN_Q_BAO_CMB'])),:].reshape((-1, obigravity_chains['SN_Q_BAO_CMB'].get_chain().shape[-1]))
corner(samples[:,4:8], levels=(1-np.exp(-0.5*1),1-np.exp(-0.5*4)),
            labels=labs[4:8], smooth=True, smooth1d=True, bins=25, plot_datapoints=False, color='cyan', 
            fill_contours=True, contourf_kwargs=dict(colors=['white', 'lightblue', 'cyan'], alpha=0.4),
            fig=fig, range=[(0,.6), (0,1.25), (0.03,0.07), (60,75)])

samples = obigravity_chains['BAO'].get_chain()[Nmin:,np.delete(range(nwalkers), calcdellist(obigravity_chains['BAO'])),:].reshape((-1, obigravity_chains['BAO'].get_chain().shape[-1]))
fig=corner(samples[:,4:8], levels=(1-np.exp(-0.5*1),1-np.exp(-0.5*4)),
            labels=labs[4:8], smooth=True, smooth1d=True, bins=25, plot_datapoints=False, color='red', 
            fill_contours=True, contourf_kwargs=dict(colors=['white', 'pink', 'red'], alpha=0.4),
            fig=fig, range=[(0,1), (0,1.25), (0.03,0.07), (60,75)])


plt.tight_layout()
plt.savefig('plots/posterior_obigravity.pdf')


plt.show()

# Model 5: Conformal Gravity

## which data sets to include?

In [ ]:
model = 'conformal'
CMBdata = CMB_data(model, 'Planck18')

In [ ]:
data_sets = [SNdata, Qdata, BAOdata]
data_set_names = [set.name for set in data_sets]
data_sets_str = '_' + '_'.join([name for name in data_set_names]) + '_' 

## Define the likelihood and priors

In [ ]:
ranges_conformal_min=[0, 0, 0, 60, -5, -10, -30, -.5, 0, 0]
ranges_conformal_max=[1, 100, 0.1, 80, 5, 10, -10, .5, 10, 3]
def lnprob_conformal(theta):
    l = likelihood(theta, data_sets, ranges_conformal_min, ranges_conformal_max, model = 'conformal', rd_num = rd_num, z_num = z_num)
    return l.logprobability_gauss_prior()

## define a reference BIC

In [ ]:
BIC_conformal = lambda theta, data_sets_var: np.log(sum([len(d.get_data()) for d in data_sets_var])) * len(theta) -2*lnprob_conformal(theta)

## run the MCMC sampler  [arXiv:1202.3665]...

In [ ]:
ndim, nwalkers, nsteps = len(ranges_conformal_min), 500, 1000
pos0 = np.random.uniform(ranges_conformal_min, ranges_conformal_max, (nwalkers,len(ranges_conformal_max)))

# pool = Pool(cpu_count())
# write = HDFBackend('chains/conformal_' + str(nwalkers) + 'x' + str(nsteps) + '.h5')
# sampler = EnsembleSampler(nwalkers, ndim, lnprob_conformal, pool=pool)#, backend=write)

# sampler.run_mcmc(pos0, nsteps, progress=True)

## ...or load existing chains

In [ ]:
conformal_sampler = HDFBackend('chains/conformal' + data_sets_str + '512x1000.h5', read_only=True)
# LCDM_sampler = sampler

nsteps, nwalkers, ndim = conformal_sampler.get_chain().shape

labs_k=[r'$\Omega_k$', r'$(\kappa/\mathrm{Gpc}^{-2})$', r'$\Omega_b$', r'$H_0\ [\frac{\mathrm{km}/s}{\mathrm{Mpc}}]$', r'$\alpha$', r'$\beta$', r'$M_B$', r'$\Delta M_B$', r'$\beta^\prime$', r'$\delta$']
labs=[r'$(\gamma_0/\mathrm{Gpc}^{-1})$', r'$(\kappa/\mathrm{Gpc}^{-2})$', r'$\Omega_b$', r'$H_0\ [\frac{\mathrm{km}/s}{\mathrm{Mpc}}]$', r'$\alpha$', r'$\beta$', r'$M_B$', r'$\Delta M_B$', r'$\beta^\prime$', r'$\delta$']

## Check and visualise the convergence: $\tau_f / n_\mathrm{steps} < 1/30$

In [ ]:
Nmin = 900
while Nmin < nsteps and np.all(conformal_sampler.get_autocorr_time(tol=0,discard=Nmin)/(nsteps)*30 > 1):
    Nmin+=100
    
print('Nmin = {}'.format(Nmin))

In [ ]:
conformal_samples = conformal_sampler.get_chain()
conformal_samples = conformal_samples[:,np.where(conformal_samples[0].T[0] < .5)[0],:]

dellist = np.unique(np.where(np.isclose(conformal_samples[-1] - conformal_samples[0], 0))[0])

conformal_samples = np.delete(conformal_samples, np.unique(dellist), axis=1)

f, ax = plt.subplots(len(conformal_samples.T), 1, figsize=(6,3*len(conformal_samples.T)))

for i in range(len(conformal_samples.T)):
    ax[i].plot(conformal_samples.T[i].T, lw=0.5)
    ax[i].plot([Nmin, Nmin], [min(conformal_samples.T[i].flatten()), max(conformal_samples.T[i].flatten())], c='k', lw=1.5)
    ax[i].text(Nmin, min(conformal_samples.T[i].flatten()), r'$n_\mathrm{min}$', rotation=90, ha='right')
    ax[i].set_ylabel(labs[i])
    ax[i].set_xlabel('iteration')
    
    
plt.tight_layout()
plt.savefig('plots/convergence_conformal.png')
plt.show()

In [ ]:
conformal_samples = conformal_samples[Nmin:, :].reshape((-1, ndim))

meanconformal = np.mean(conformal_samples, axis=0)
stdconformal = np.var(conformal_samples, axis=0)
maxconformal=[]
for i in range(len(meanconformal)):
    like = np.histogram(conformal_samples.T[i], bins=50)
    i_max=np.argmax(like[0])
    max_val = (like[1][i_max]+like[1][i_max+1])/2
    maxconformal.append(max_val)

    
# conformal_samples.T[0] = (conformal_samples.T[0])**2 / 2 * cLight**2/(70./1E-3)**2
fig = corner(conformal_samples, quantiles=(.16,.84),  levels=(1-np.exp(-0.5),1-np.exp(-0.5*4)),
             labels=labs, smooth=True, smooth1d=True, bins=20, plot_datapoints=False, fill_contours=True, contourf_kwargs=dict(colors=None, cmap='Greens'))#,
# #              truths=maxconformal)




plt.tight_layout()
plt.savefig('plots/posterior_conformal' + data_sets_str[:-1] + '.pdf')

In [ ]:
v = np.percentile(conformal_samples,[16, 50, 84], axis = 0)
v = np.asarray([v[1], v[2]-v[1],v[1]-v[0]]).T

gamma0, gamm0p, gamma0m = v[0]
kappa, kappap, kappam = v[1]

omegak = (gamma0)**2 / 2 * cosmology.cLight**2/(70./1E-3)**2
stdkp, stdkm = np.array([gamm0p, gamma0m]) * (gamma0) * cosmology.cLight**2/(70./1E-3)**2#Gpc^-1


omegab, stdbp, stdbm = v[2]
H0, stdhp, stdhm = v[3]

omegac = 1 - omegak
stdcp = stdkm
stdcm = stdkp
#model
z = SNdata.get_data().T[0]

best_fit_conformalcosmo = cosmology(0, omegac, omegab=omegab, omegar=omega_gamma_preset, Hzero=H0)
SNdata.set_param(v.T[0][-6:-2])
Qdata.set_param(v.T[0][-2:])

In [ ]:
bic_CG = BIC_conformal(v.T[0],data_sets)

print('BIC(CG) = ' + str(bic_CG))

In [ ]:
n = sum([len(galaxy) for galaxy in RCdata.get_data()])

In [ ]:
BIC_conformal(v.T[0],data_sets) - 141759 + n*np.log(n)

In [ ]:
bic_lcdmlist

In [ ]:
fig = plt.figure(figsize=(8,8))
gs = gridspec.GridSpec(3, 1)
gs.update(hspace=0)


ax1 = plt.subplot(gs[0:2, 0])

if 'RC' in data_set_names:
    ax1.set_title(r'$\Omega_\Lambda = {%1.3f}^{+%1.3f}_{-%1.3f},\, 1000\,\Omega_k = {%1.3f}^{+%1.3f}_{-%1.3f},\, H_0 =  {%1.1f}^{+%1.1f}_{-%1.1f}\,\frac{\mathrm{km}/s}{\mathrm{Mpc}}$' % (omegac, stdcp,stdcm, 1000*omegak, 1000*stdkm, 1000*stdkp, H0, stdhp, stdhm), fontsize=18)
else:
    ax1.set_title(r'$\Omega_\Lambda = {%1.3f}^{+%1.3f}_{-%1.3f},\, \Omega_k = {%1.3f}^{+%1.3f}_{-%1.3f},\, H_0 =  {%1.1f}^{+%1.1f}_{-%1.1f}\,\frac{\mathrm{km}/s}{\mathrm{Mpc}}$' % (omegac, stdcp,stdcm, omegak, stdkm, stdkp, H0, stdhp, stdhm), fontsize=18)

if 'Quasars' in data_set_names:
    ax1.errorbar(Qdata.distance_modulus().T[0], Qdata.distance_modulus().T[1], yerr=Qdata.delta_distance_modulus(), linestyle='none', marker='o', color='orange', ecolor='yellow', markersize=3, alpha=0.6, zorder=-1, label=r'quasars')
if 'SN' in data_set_names:
    ax1.errorbar(SNdata.distance_modulus().T[0], SNdata.distance_modulus().T[1], yerr=SNdata.delta_distance_modulus(), linestyle='none', marker='o', color='cyan', ecolor='blue', markersize=3, alpha=0.6, zorder=-1, label=r'SNe')
if 'BAO' in data_set_names:
    ax1.errorbar(BAOdata.distance_modulus(best_fit_conformalcosmo).T[0], BAOdata.distance_modulus(best_fit_conformalcosmo).T[1], yerr=BAOdata.delta_distance_modulus(best_fit_conformalcosmo), linestyle='none', marker='o', color='red', ecolor='black', markersize=3, alpha=0.6, zorder=1, label=r'BAO')



zPlot = np.logspace(-3,np.log10(6),100)
ax1.plot(zPlot, best_fit_conformalcosmo.distance_modulus(zPlot), c='k', label=r'best fit')
ax1.fill_between(zPlot, cosmology(0,omegac+stdcp, omegab=omegab+stdbp, Hzero=H0 + stdhp).distance_modulus(zPlot), cosmology(0,omegac-stdcm, omegab=omegab-stdbm, Hzero=H0 - stdhm).distance_modulus(zPlot), color='gray', alpha=0.3)
ax1.plot(zPlot, cosmology(Omega_m_preset, Omega_c_preset).distance_modulus(zPlot), ls = '--', c='gray', label=r'$\Lambda$CDM')

ax1.set_xlim(0.01,6)
ax1.set_ylim(32,55)


ax1.set_ylabel(r'distance modulus $\mu$')
ax1.set_xticklabels([])

ax1.grid('--', dashes=(5,5))
ax1.legend()


ax2 = plt.subplot(gs[2, 0])

ax2.plot(zPlot, best_fit_conformalcosmo.distance_modulus(zPlot) - cosmology(Omega_m_preset, Omega_c_preset).distance_modulus(zPlot), 'k')

# ax2.set_xscale('log')
ax2.set_xlim(0.01,6)

ax2.text(0.97, .06, r'$\Delta\mathrm{BIC}(\mathrm{CG}) = %1.0f$' % (BIC_conformal(v.T[0],data_sets) - bic_lcdmlist['SN_Q_BAO']), ha='right', va='bottom', transform=ax2.transAxes, bbox=dict(facecolor=(1,1,1,.8), edgecolor='lightgray', boxstyle='round'))


ax2.grid('--', dashes=(5,5))
ax2.set_xlabel(r'$z$')
ax2.set_ylabel(r'$\mu_\mathrm{fit} - \mu_{\Lambda\mathrm{CDM}}$')

plt.tight_layout()
plt.savefig('plots/Hubble_conformal' + data_sets_str[:-1] + '.pdf')
plt.show()

## Joint fit of all data sets

In [ ]:
conformal_chains = {}
conformal_chains['SN'] = HDFBackend('chains/conformal_SN_512x1000.h5')
conformal_chains['SN_Q'] = HDFBackend('chains/conformal_SN_Quasars_512x1000.h5')
conformal_chains['BAO'] = HDFBackend('chains/conformal_BAO_512x1000.h5')
# conformal_chains['CMB'] = HDFBackend('chains/conformal_CMB_512x1000.h5')
conformal_chains['SN_Q_BAO'] = HDFBackend('chains/conformal_SN_Quasars_BAO_512x1000.h5')
#conformal_chains['RC'] = HDFBackend('chains/conformal_RC_512x1000.h5')
#conformal_chains['SN_RC'] = HDFBackend('chains/conformal_RC_SN_512x1000.h5')
#conformal_chains['SN_Q_RC'] = HDFBackend('chains/conformal_RC_SN_Quasars_512x1000.h5')
#conformal_chains['SN_Q_BAO_RC'] = HDFBackend('chains/conformal_RC_SN_Quasars_BAO_512x1000.h5')

In [ ]:
data_set_combinations = ['SN','SN_Q','SN_Q_BAO']
data_set_combin_obs = {}
data_set_combin_obs['SN'] = [SNdata]
data_set_combin_obs['SN_Q'] = [SNdata,Qdata]
data_set_combin_obs['SN_Q_BAO'] = [SNdata,Qdata,BAOdata]
#data_set_combin_obs['SN_BAO_CMB'] = [SNdata,BAOdata,CMBdata]
#data_set_combin_obs['SN_Q_BAO_CMB'] = [SNdata,Qdata,BAOdata,CMBdata]
#data_set_combin_obs['BAO'] = [BAOdata]
#data_set_combin_obs['CMB'] = [CMBdata]

In [ ]:
for set_name in data_set_combinations:
    sample = conformal_chains[set_name].get_chain()
    Nmin = 900
    while Nmin < nsteps and np.all(conformal_chains[set_name].get_autocorr_time(tol=0,discard=Nmin)/(nsteps)*30 > 1):
        Nmin+=100
    dellist = np.unique(np.where(np.isclose(sample[-1] - sample[0], 0))[0])
    sample = np.delete(sample, np.unique(dellist), axis=1)[Nmin:, :].reshape((-1, ndim))
    
    data_sets = data_set_combin_obs[set_name]
    
    vtemp = np.percentile(sample,[16, 50, 84], axis = 0)
    vtemp = np.asarray([vtemp[1], vtemp[2]-vtemp[1],vtemp[1]-vtemp[0]]).T
    
    gamma0, gamm0p, gamma0m = vtemp[0]

    omegak = (gamma0)**2 / 2 * cosmology.cLight**2/(70./1E-3)**2
    stdkp, stdkm = np.array([gamm0p, gamma0m]) * (gamma0) * cosmology.cLight**2/(70./1E-3)**2#Gpc^-1

    print(set_name + ':')
    print('   Om_k:   ' + str(np.round([omegak,stdkp,stdkm],4)))
    print('   Om_b:   ' + str(np.round(vtemp[2],4)*100))
    print('   kappa:  ' + str(np.round(vtemp[1],4)))
    print('   H0:     ' + str(np.round(vtemp[3],1)))
    print('   dBIC:   ' + str(np.round(BIC_conformal(vtemp.T[0], data_set_combin_obs[set_name])-bic_lcdmlist[set_name])))
    #print(vtemp.T[0])

In [ ]:
conformal_chains

In [ ]:
Nmin = 900
for sampler in conformal_chains.values():
    nsteps, nwalkers, ndim = sampler.get_chain().shape
    while Nmin < nsteps and np.all(sampler.get_autocorr_time(tol=0,discard=Nmin)/(nsteps)*30 > 1):
        Nmin+=100
print('Nmin=',Nmin)

### No Rot curves -> exclude $\kappa$

In [ ]:
samples = conformal_chains['SN'].get_chain()[Nmin:,np.delete(range(nwalkers), calcdellist(conformal_chains['SN'])),:].reshape((-1, conformal_chains['SN'].get_chain().shape[-1]))
samples = samples[np.where(samples.T[0] < .33)[0]]
samples.T[0] = (samples.T[0])**2 / 2 * cLight**2/(H0/1E-3)**2
samples = np.array([samples.T[i] for i in (0,2,3)]).T
fig=corner(samples, levels=(1-np.exp(-0.5),1-np.exp(-0.5*4)),
            labels=np.delete(labs_k, 1), smooth=True, smooth1d=True, bins=25, plot_datapoints=False, color='green',
            fill_contours=True, contourf_kwargs=dict(colors=['white', 'lightgreen', 'green']),
            range=[(0,1), (0, .1), (62,75)])

samples = conformal_chains['SN_Q'].get_chain()[Nmin:,np.delete(range(nwalkers), calcdellist(conformal_chains['SN_Q'])),:].reshape((-1, conformal_chains['SN_Q'].get_chain().shape[-1]))
samples = samples[np.where(samples.T[0] < .33)[0]]
samples.T[0] = (samples.T[0])**2 / 2 * cLight**2/(H0/1E-3)**2
samples = np.array([samples.T[i] for i in (0,2,3)]).T
corner(samples, levels=(1-np.exp(-0.5),1-np.exp(-0.5*4)), color='blue',
            labels=np.delete(labs_k, 1), smooth=True, smooth1d=True, bins=25, plot_datapoints=False, 
            fill_contours=True, contourf_kwargs=dict(colors=['white', 'lightblue', 'blue'], alpha=0.6),
            fig=fig, range=[(0,1), (0, .1), (62,75)])


samples = conformal_chains['BAO'].get_chain()[Nmin:,np.delete(range(nwalkers), calcdellist(conformal_chains['BAO'])),:].reshape((-1, conformal_chains['BAO'].get_chain().shape[-1]))
samples = samples[np.where(samples.T[0] < .33)[0]]
samples.T[0] = (samples.T[0])**2 / 2 * cLight**2/(H0/1E-3)**2
samples = np.array([samples.T[i] for i in (0,2,3)]).T
fig=corner(samples, levels=(1-np.exp(-0.5*1),1-np.exp(-0.5*4)),
            labels=np.delete(labs_k, 1), smooth=True, smooth1d=True, bins=25, plot_datapoints=False, color='red', 
            fill_contours=True, contourf_kwargs=dict(colors=['white', 'pink', 'red'], alpha=0.4),
            fig=fig, range=[(0,1), (0, .1), (62,75)])


# samples = conformal_chains['CMB'].get_chain()[Nmin:,:].reshape((-1, conformal_chains['CMB'].get_chain().shape[-1]))
# samples = samples[np.where(samples.T[0] < .33)[0]]
# samples = np.array([samples.T[i] for i in (0,2,3)]).T
# corner(samples, levels=(1-np.exp(-0.5*1),1-np.exp(-0.5*4)),
#             labels=np.delete(labs_k, 1), smooth=True, smooth1d=True, bins=25, plot_datapoints=False, color='orange', 
#             fill_contours=True, contourf_kwargs=dict(colors=['white', 'yellow', 'orange'], alpha=0.4),
#             fig=fig)#, range=[(0,1), (50,300), (0, .1), (62,75)])

samples = conformal_chains['SN_Q_BAO'].get_chain()[Nmin:,np.delete(range(nwalkers), calcdellist(conformal_chains['SN_Q_BAO'])),:].reshape((-1, conformal_chains['SN_Q_BAO'].get_chain().shape[-1]))
samples = samples[np.where(samples.T[0] < .33)[0]]
samples.T[0] = (samples.T[0])**2 / 2 * cLight**2/(H0/1E-3)**2
samples = np.array([samples.T[i] for i in (0,2,3)]).T
corner(samples, levels=(1-np.exp(-0.5*1),1-np.exp(-0.5*4)),
            labels=np.delete(labs_k, 1), smooth=True, smooth1d=True, bins=25, plot_datapoints=False, color='cyan', 
            fill_contours=True, contourf_kwargs=dict(colors=['white', 'lightblue', 'cyan'], alpha=0.4),
            fig=fig, range=[(0,1), (0, .1), (62,75)])



plt.tight_layout()
plt.savefig('plots/posterior_conformal.pdf')

plt.show()

### With Rot curves -> include $\kappa$ :
Because CG is already severely restricted without RC, we do not eveninclude this dataset

In [ ]:
labs_k[0] = r'$\log_{10}(\Omega_k)$'

samples = conformal_chains['SN_Q_BAO'].get_chain()[Nmin:,:].reshape((-1, conformal_chains['SN_Q_BAO'].get_chain().shape[-1]))
samples = samples[np.where(samples.T[0] < .33)[0]]
samples = np.array([samples.T[i] for i in (0,1)]).T
samples.T[0] = np.log10((samples.T[0])**2 / 2 * cLight**2/(H0/1E-3)**2)
fig=corner(samples, levels=(1-np.exp(-0.5*1),1-np.exp(-0.5*4)),
            labels=labs_k, smooth=True, smooth1d=True, bins=25, plot_datapoints=False, color='cyan', 
            fill_contours=True, contourf_kwargs=dict(colors=['white', 'lightblue', 'cyan'], alpha=0.4),
            range=[(-3,0), (50,250)])

samples = conformal_chains['RC'].get_chain()[Nmin:,:].reshape((-1, conformal_chains['RC'].get_chain().shape[-1]))
samples = samples[np.where(samples.T[0] < .33)[0]]
samples = np.array([samples.T[i] for i in (0,1)]).T
samples.T[0] = np.log10((samples.T[0])**2 / 2 * cLight**2/(H0/1E-3)**2)
corner(samples, levels=(1-np.exp(-0.5),1-np.exp(-0.5*4)),
            labels=labs_k, smooth=True, smooth1d=True, bins=25, plot_datapoints=False, color='orange',
            fill_contours=True, contourf_kwargs=dict(colors=['white', 'yellow', 'orange']),
            fig=fig, range=[(-3,0), (50,250)])


samples = conformal_chains['SN_RC'].get_chain()[Nmin:,:].reshape((-1, conformal_chains['SN_RC'].get_chain().shape[-1]))
samples = samples[np.where(samples.T[0] < .33)[0]]
samples = np.array([samples.T[i] for i in (0,1)]).T
samples.T[0] = np.log10((samples.T[0])**2 / 2 * cLight**2/(H0/1E-3)**2)
corner(samples, levels=(1-np.exp(-0.5),1-np.exp(-0.5*4)),
            labels=labs_k, smooth=True, smooth1d=True, bins=25, plot_datapoints=False, color='green',
            fill_contours=True, contourf_kwargs=dict(colors=['white', 'lightgreen', 'green']),
            fig=fig, range=[(-3,0), (50,250)])

samples = conformal_chains['SN_Q_RC'].get_chain()[Nmin:,:].reshape((-1, conformal_chains['SN_Q_RC'].get_chain().shape[-1]))
samples = samples[np.where(samples.T[0] < .33)[0]]
samples = np.array([samples.T[i] for i in (0,1)]).T
samples.T[0] = np.log10((samples.T[0])**2 / 2 * cLight**2/(H0/1E-3)**2)
corner(samples,  levels=(1-np.exp(-0.5),1-np.exp(-0.5*4)), color='blue',
            labels=labs_k, smooth=True, smooth1d=True, bins=25, plot_datapoints=False, 
            fill_contours=True, contourf_kwargs=dict(colors=['white', 'lightblue', 'blue'], alpha=0.6),
            fig=fig, range=[(-3,0), (50,250)])

samples = conformal_chains['SN_Q_BAO_RC'].get_chain()[Nmin:,:].reshape((-1, conformal_chains['SN_Q_BAO_RC'].get_chain().shape[-1]))
samples = samples[np.where(samples.T[0] < .33)[0]]
samples = np.array([samples.T[i] for i in (0,1)]).T
samples.T[0] = np.log10((samples.T[0])**2 / 2 * cLight**2/(H0/1E-3)**2)
corner(samples, levels=(1-np.exp(-0.5),1-np.exp(-0.5*4)), color='red',
            labels=labs_k, smooth=True, smooth1d=True, bins=25, plot_datapoints=False, 
            fill_contours=True, contourf_kwargs=dict(colors=['white', 'pink', 'red'], alpha=0.6),
            fig=fig, range=[(-3,0), (50,250)])




plt.tight_layout()
plt.savefig('plots/posterior_conformal_RC.pdf')

plt.show()

## Rotation curves

In [ ]:
gamma0, kappa = v[0:2,0]
print(gamma0, kappa)

In [ ]:
GN =  4.301E3 #GN * 10^9 Msol / 1kpc in (km/s)^2

def vhalo_square(r, Ms, rs): # units of 1E9 Msol
    return GN * Ms * (np.log(1+r/rs) - r/(r+rs))/r

def vCG_square(r, gamma0=gamma0, kappa=kappa): # units of 1E9 Msol
    r = r / 1E6 #Gpc
    return cLight**2 * gamma0 * r / 2 - kappa * cLight**2 * r**2

def vlocal_square (r, r0, M0, gamma):
        return GN * M0 * (r/r0)**2 * (i0(r/2/r0) * k0(r/2/r0) - i1(r/2/r0) * k1(r/2/r0)) + gamma * (r/r0)**2 * i1(r/2/r0) * k1(r/2/r0)

In [ ]:
f, ax = plt.subplots(int(np.ceil(len(RCdata.get_data())/5)),5, figsize=(12,2.5*len(RCdata.get_data())/5))
# f, ax = plt.subplots(1,5, figsize=(15,4))

ax = ax.flatten()

pDMList=[]
chiDM=[]
pCGList=[]
chiCG=[]

galaxies = RCdata.get_data()
names = RCdata.get_names()

for i in range(len(galaxies)):
    visible = lambda r, YD, YB: interp1d(galaxies[i][:,0], galaxies[i][:,3]**2 + YD*galaxies[i][:,4]**2 + YB*galaxies[i][:,5]**2, kind='cubic')(r)
    fitf_DM =  lambda r, YD, YB, Ms, rs,: visible(r, YD, YB) + vhalo_square(r, Ms,rs) 
    pDM, _ = curve_fit(fitf_DM, galaxies[i][:,0], galaxies[i][:,1]**2, sigma=galaxies[i][:,2]**2, p0=[1,1, 5, 2*max(galaxies[i][:,0])], bounds = ([.1,.1,0,1], [5,5,250,10*max(galaxies[i][:,0])]))
    
    
    pGas, _ = curve_fit(vlocal_square,  galaxies[i][:,0], galaxies[i][:,3]**2, p0=[1,10,1], bounds=(0, 500))
    pDisk, _ = curve_fit(vlocal_square,  galaxies[i][:,0], galaxies[i][:,4]**2, p0=[1,10,1], bounds=(0, 500))
    visibleCG = lambda r, YD, YB: vlocal_square(r, *pGas) + YD * vlocal_square(r, *pDisk) + YD * interp1d(galaxies[i][:,0], galaxies[i][:,5]**2, kind='cubic')(r)
    fitf_CG = lambda r, YD, YB: visibleCG(r, YD, YB) + vCG_square(r) 
    pCG, _ = curve_fit(fitf_CG, galaxies[i][:,0], galaxies[i][:,1]**2, sigma=galaxies[i][:,2]**2, p0=[1, 1])#, bounds = ([0.1,.1,0,0], [5,5,10,10]))

    pDMList.append(pDM)
    pCGList.append(pCG)
    chiDM.append(np.sum( (galaxies[i][:,1] - fitf_DM(galaxies[i][:,0],*pDM)**.5)**2 / galaxies[i][:,2]**2))
    chiCG.append(np.sum( (galaxies[i][:,1] - fitf_CG(galaxies[i][:,0],*pCG)**.5)**2 / galaxies[i][:,2]**2))
    
    
    ax[i].set_title(names[i])
    ax[i].set_xlabel(r'$r\ [\mathrm{kpc}]$')
    ax[i].set_ylabel(r'$v\ [\mathrm{km}/ \mathrm{s}]$')
    ax[i].plot(galaxies[i][:,0], galaxies[i][:,3], c='lightgreen', lw=0.6, label='gas')
    ax[i].plot(galaxies[i][:,0], np.sqrt(pDM[0])*galaxies[i][:,4], c='blue', lw=0.6, label='disk')
    ax[i].plot(galaxies[i][:,0], np.sqrt(pDM[1])*galaxies[i][:,5], c='m', lw=0.6, label='bulge')
    ax[i].plot(galaxies[i][:,0], vhalo_square(galaxies[i][:,0], *pDM[2:])**.5, c='maroon', ls=':') 
    ax[i].plot(galaxies[i][:,0], vCG_square(galaxies[i][:,0])**.5, c='y', ls='-.') 
    ax[i].plot(galaxies[i][:,0], np.sqrt(galaxies[i][:,3]**2 + pDM[0]*galaxies[i][:,4]**2 + pDM[1]*galaxies[i][:,5]**2 + vhalo_square(galaxies[i][:,0], *pDM[2:])), c='r')
    ax[i].plot(galaxies[i][:,0], np.sqrt(galaxies[i][:,3]**2 +  pCG[0]*galaxies[i][:,4]**2 +  pCG[1]*galaxies[i][:,5]**2+ vCG_square(galaxies[i][:,0], *pCG[2:])), c='orange')
    ax[i].errorbar(galaxies[i][:,0], galaxies[i][:,1], yerr=galaxies[i][:,2], c='k', fmt='o', markersize=3)
    
    
plt.tight_layout()
# plt.savefig('plots/SPARC_DM.pdf')
plt.show()

In [ ]:
bic_CG_RT = np.sum(np.array(chiCG)[~np.isnan(chiCG)])
# bic_CG_RT += (3*n + 2)*np.log(n)
bic_CG_RT

In [ ]:
bic_lcdm_RT =  np.sum(np.array(chiDM)[~np.isnan(chiCG)])
# bic_lcdm_RT += 4*n*np.log(n)
bic_lcdm_RT

In [ ]:
(bic_CG_RT - bic_lcdm_RT)